# Load tables and funcitons

In [1]:
# this file creates the DF for the material flow Sankey Diagram
#load the tables, 


In [2]:
## Upload libraries
import sys
import pandas as pd
import numpy as np

#upload HY funcitons
from Functions.calc_func import * 
from Functions.upload_functions import * 


# export file path:
expath = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/'
tests = 'tests/'
ca_material_lib = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/CA_materials_GFCFC_df/'

# Canada Province list s loaded.
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
provinces

sector = 'Gross fixed capital formation, Construction'


def attach_mat_class(emission_df):
    '''This function attaches the material classificaiton to a chosen enission df
    - updated emission classificaiton is material_lca_classification09.csv '''

    # # 1. import calssification
    path = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/lists/'
    class_list = pd.read_csv(path + 'material_lca_classification09.csv')
    # arrange by country/commodity
    class_list.set_index(['country_province', 'commodity' ])

    # removes multi-index from LCA DF     left = on_res_commod_LCA.droplevel([1], axis = 1)
    left = emission_df
    #Creating a refernce index for both DF (country+commodity)
    left = left.reset_index()
    left['new_index'] = left['Unnamed: 0'] + ' ' + left['Unnamed: 1']
    left = left.set_index('new_index')
    left

    right = class_list
    right['new_index'] = right['country_province'] + ' ' + right['commodity']
    right = right.set_index('new_index')
    right

    #Joining DF
    material_df = left.join(right)
    return material_df

def sum_odd_even_columns_with_original_names(dataframe):
    columns = dataframe.columns
    sum_df = pd.DataFrame()

    for i in range(0, len(columns), 2):
        if i + 1 < len(columns):
            col_name = f'Sum_{columns[i]}'
            sum_series = dataframe[columns[i]] + dataframe[columns[i + 1]]
            sum_df[col_name] = sum_series

    return sum_df

# 0.0 calculate X-Total Output from tables 

In [3]:
## IO Tables. (CIRAIG, CanadaIO V2.4, 2022) 2018 Statistics Canada data

# A	The technology matrix (16196 rows × 16196 columns) shows the normalized inter-industry trade. Read the information as "To produce 1$ of the product from the column, I need X$ of the product from the row"																	
															
# Y	The final demand matrix (16196 rows × 3627 columns) shows the demands from the different end-users. Read the information as "The consumer category from the column has bought X$ of the product in the row during the given year"																			
																				
# R	R (104 rows × 6396 columns) gives the factors of production (taxes, value-added, subsidies, etc.). Read the information as "To produce 1$ of the product from the column, I need X$ of factors of production from the row"																			
																				
# S	(11085 rows × 16196 columns) The normalized environmental extensions give the normalized direct emissions for each product. Read the information as "To produce 1$ of the product from the column, I emit Xkg of the pollutant from the row"																			
																				
# FY The direct emissions from the final demand gives the direct emissions from the different end-users, which are thus not directly linked to an economic activity (e.g., burning diesel in one's car). Read the information as "The consumer category from the column has directly emitted Xkg of pollutant from the row during the given year"																			
																				
# C	(33 rows × 11085 columns) The characterization matrix converts emissions of pollutants to impacts on the environment. Read the information as "The pollutant from the column has an impact of X on the impact category from the row"																			
																				
# L (16196 rows × 16196 columns) Liontief matrix

# FY (11085 rows × 3627 columns) - Direct emissions

In [7]:
#This cell loads all od CIRAIG's IO tables
# Tables use 2018 Stat Can. IO data, and openIO V2.4 EEIO model

#load Y table - final demand matrix (16196 rows × 3627)
#make sure to assign columns as indexes and headars, if not deafault

tables_Y = machine_loc('craig_Yfinal_demand.csv')
# load R matrix (104 rows × 6396 columns)
#tables_R = machine_loc('craig_R_norm_tech_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load A matrix (16196 rows × 16196 columns)
tables_A = machine_loc('craig_A_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load L matrix (16196 rows × 16196 columns)
tables_L = machine_loc('craig_L_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load S matrix (11085 rows × 16196 columns)
tables_S = machine_loc('craig_S_norm_env_flow_matrix.csv',index_colf = [0], headerf = [0,1])
#load C matrix (33 rows × 11085 columns)
tables_C = machine_loc('craig_C_characterization_matrix.csv',index_colf = [0,1], headerf = 0)
#load FY matrix (11085 rows × 3626 columns)
#tables_FY = machine_loc('craig_FY_direct_emissions_matrix.csv',index_colf = [0,1], headerf = [0,1,2])

# calculate (C)(S) table - environmental impact
tables_CS = tables_C.dot(tables_S)
print('Table CS calculated successfuly')

print('\n All tables loaded successfuly')

Couldnt load file on VM
loading craig_Yfinal_demand.csv from hard drive...
craig_Yfinal_demand.csv loaded successfuly
Couldnt load file on VM
loading craig_A_matrix.csv from hard drive...


KeyboardInterrupt: 

## 0.1 Calculating gfcfc sub-sector ('construction')

In [ ]:
# Designate final demand sector
# this demo uses ('CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Construction'

# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector, sub_sector)
final_demand_category

# isolate final demnad vector

on_gfcf_sub_Y = tables_Y[(final_demand_category)]
#transform series into df
on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
on_gfcf_sub_Y

# 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
on_gfcf_sub_AY= vector_X(tables_A, on_gfcf_sub_Y)
#call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df

# This line multiplies (L)16Kx16K * (AYdiag) 16Kx16K - the intermediate output

on_gfcf_sub_D= vector_D(tables_L, on_gfcf_sub_AY[1])
on_gfcf_sub_D

def sum_col(df):


    # Get the column names
    column_names = df.columns

    # Initialize a new DataFrame to store the summed values
    summed_df = pd.DataFrame()

    # Iterate through even columns (0, 2, 4, etc.) and sum with the following odd column
    for i in range(0, len(column_names) - 1, 2):
        even_column = column_names[i]
        odd_column = column_names[i + 1]
        
        # Sum the even and odd columns and store the result in the new DataFrame
        #summed_df[f'{even_column} + {odd_column}'] = df[even_column] + df[odd_column]
        summed_df[even_column] = df[even_column] + df[odd_column]


    # Display the resulting DataFrame
    return summed_df

x = sum_col(ca_gfcfc_mat_emission)


(                                                                              CA-AB  \
                                                         Canola (including rapeseed)   
 CA-AB Canola (including rapeseed)                                               0.0   
       Oilseeds (except canola)                                                  0.0   
       Wheat                                                                     0.0   
       Grains (except wheat)                                                     0.0   
       Fresh potatoes                                                            0.0   
 ...                                                                             ...   
 WM    Membership organisation services n.e.c. (91)                              0.0   
       Recreational, cultural and sporting services (92)                         0.0   
       Other services (93)                                                       0.0   
       Private households with e

In [ ]:
on_gfcf_sub_Y.sum()

CA-ON  Gross fixed capital formation, Construction  Construction    3.365139e+08
dtype: float64

In [ ]:
#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
on_gfcf_sub_CSD = env_impact(tables_CS, on_gfcf_sub_D[0])

#3. assign title to column
on_gfcf_sub_CSD = pd.DataFrame(on_gfcf_sub_CSD).rename(
    columns = {'Climate change, short term': ('on_gfcfc_'+sub_sector)})
on_gfcf_sub_CSD


on_gfcfc_Construction
                                                             kg CO2 eq (short)
CA-ON Ready-mixed concrete                                        9.724398e+06
      Concrete products                                           4.663728e+06
CN    Electrical machinery and apparatus n.e.c. (31)              2.407368e+06
      Fabricated metal products, except machinery and...          2.241158e+06
CA-ON Non-metallic mineral products, n.e.c.                       1.894882e+06
...                                                                        ...
CA-SK Motor vehicle metal stampings                               0.000000e+00
      Motor vehicle brakes and brake systems                      0.000000e+00
      Motor vehicle steering and suspension components            0.000000e+00
      Motor vehicle electrical and electronic equipment           0.000000e+00
WM    Extra-territorial organizations and bodies                  0.000000e+00

[16196 rows x 1 columns]

In [ ]:
#4. calculating final demand emissions LCA const. process (A4-A5). (CS)(Y)diag
#4.1 diagonolize Y
on_gfcf_sub_Ydiag = pd.DataFrame(np.diag(on_gfcf_sub_Y.sum(1)), index=on_gfcf_sub_Y.index, columns= on_gfcf_sub_Y.index)
#on_res_struct_Ydiag
#multiply final demand vector with env impact.
on_gfcf_sub_CSY = env_impact(tables_CS, on_gfcf_sub_Ydiag)
on_gfcf_sub_CSY = pd.DataFrame(on_gfcf_sub_CSY).rename(
    columns = {'Climate change, short term':'on_gfcfc_'+sub_sector})
#on_res_struct_CSY.sum()
#this line tests the intermediate output and final demand sum up to the sector's total emissions.
#on_res_struct_CSD.sum()+on_res_struct_CSY.sum()
on_gfcf_sub_CSY

on_gfcfc_Construction
                                                             kg CO2 eq (short)
CA-ON Industrial buildings (except mine buildings)                2.892381e+06
      Office buildings                                            1.543797e+06
      Real estate brokerage and other services relate...          5.617396e+05
CA-QC Real estate brokerage and other services relate...          1.796002e+04
CA-MB Real estate brokerage and other services relate...          2.098343e+03
...                                                                        ...
CA-QC Other federal government services                           0.000000e+00
      Other provincial and territorial government ser...          0.000000e+00
      Other municipal government services                         0.000000e+00
      Other aboriginal government services                        0.000000e+00
WM    Extra-territorial organizations and bodies                  0.000000e+00

[16196 rows x 1 columns]

In [ ]:
#5. Join vectors and into DF.
left = on_gfcf_sub_CSD
right = on_gfcf_sub_CSY
on_gfcf_sub_commod_LCA = left.join(right, lsuffix = '_A1-A4_intermediate_output', rsuffix = '_A5_final_demand')

on_gfcf_sub_commod_LCA

on_gfcfc_Construction_A1-A4_intermediate_output  \
                                                                                       kg CO2 eq (short)   
CA-ON Ready-mixed concrete                                                                  9.724398e+06   
      Concrete products                                                                     4.663728e+06   
CN    Electrical machinery and apparatus n.e.c. (31)                                        2.407368e+06   
      Fabricated metal products, except machinery and...                                    2.241158e+06   
CA-ON Non-metallic mineral products, n.e.c.                                                 1.894882e+06   
...                                                                                                  ...   
CA-SK Motor vehicle metal stampings                                                         0.000000e+00   
      Motor vehicle brakes and brake systems                                                0.000000e+00   
      Motor vehicle steering and suspension components                                      0.000000e+00   
      Motor vehicle electrical and electronic equipment                                     0.000000e+00   
WM    Extra-territorial organizations and bodies                                            0.000000e+00   

                                                         on_gfcfc_Construction_A5_final_demand  
                                                                             kg CO2 eq (short)  
CA-ON Ready-mixed concrete                                                                 0.0  
      Concrete products                                                                    0.0  
CN    Electrical machinery and apparatus n.e.c. (31)                                       0.0  
      Fabricated metal products, except machinery and...                                   0.0  
CA-ON Non-metallic mineral products, n.e.c.                                                0.0  
...                                                                                        ...  
CA-SK Motor vehicle metal stampings                                                        0.0  
      Motor vehicle brakes and brake systems                                               0.0  
      Motor vehicle steering and suspension components                                     0.0  
      Motor vehicle electrical and electronic equipment                                    0.0  
WM    Extra-territorial organizations and bodies                                           0.0  

[16196 rows x 2 columns]

In [ ]:
on_gfcf_sub_commod_LCA.to_csv(expath + tests + 'on_construction_commod_LCA_blue.csv')

# 1.0 Shortcut to upload material lists

In [2]:
#load table (on_res_commod_LCA) without recalculating EEIO
data_path  ='C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/'
on_res_sub_commod_LCA = pd.read_csv(data_path+ 'on_res_commod_LCA_blue.csv', header=[0, 1], index_col=[0,1])
on_res_sub_commod_LCA
#join materials to sub sector
on_res_sub_materials = join_material_index(on_gfcf_sub_commod_LCA)
on_res_sub_materials[0]
#Sort by material
x= on_res_sub_materials[1]
x.sort_values(by = ['lev2_name'])


NameError: name 'on_gfcf_sub_commod_LCA' is not defined

In [30]:
x = on_res_sub_materials[0]
x = x.sort_values(['lev1_name', 'lev2_name']) 
#x.loc['Other'] #, 'lev2_name']]
#x.sort_values(by=['lev1_name', 'lev2_name'])
x


,level_0,level_1,"A1-A4_intermediate_output_('CA-ON', 'Gross fixed capital formation, Construction', 'Residential structures')_kg CO2 eq (short)","A5_final_demand_('CA-ON', 'Gross fixed capital formation, Construction', 'Residential structures')_kg CO2 eq (short)",country_province,commodity,lev1,lev1_name,lev2,lev2_name,LCA
new_index,,,,,,,,,,,
CA-ON Repair construction services,CA-ON,Repair construction services,1.505272e+07,0.000000,CA-ON,Repair construction services,5,ConstructionSector,05 02,DirectEmissions,A5
CA-QC Repair construction services,CA-QC,Repair construction services,1.148302e+06,0.000000,CA-QC,Repair construction services,5,ConstructionSector,05 02,DirectEmissions,A5
US Real estate services (70),US,Real estate services (70),2.234308e+05,141.396155,US,Real estate services (70),5,ConstructionSector,05 02,DirectEmissions,0
CA-MB Repair construction services,CA-MB,Repair construction services,1.047250e+05,0.000000,CA-MB,Repair construction services,5,ConstructionSector,05 02,DirectEmissions,A5
WA Construction work (45),WA,Construction work (45),2.555217e+04,0.000000,WA,Construction work (45),1,ConstructionSector,05 02,DirectEmissions,0
...,...,...,...,...,...,...,...,...,...,...,...
BE Steam and hot water supply services,BE,Steam and hot water supply services,0.000000e+00,0.000000,BE,Steam and hot water supply services,2,Water,02 01,Unit,0
CA-NU Steam and heated or cooled air or water,CA-NU,Steam and heated or cooled air or water,0.000000e+00,0.000000,CA-NU,Steam and heated or cooled air or water,2,Water,02 01,Unit,A1-A3
IN Steam and hot water supply services,IN,Steam and hot water supply services,0.000000e+00,0.000000,IN,Steam and hot water supply services,2,Water,02 01,Unit,0


In [40]:
lev1_other = x.loc[x.lev1_name == 'Other'] 
lev1_other.set_index(['lev1_name', 'lev2_name']).head(2)


level_0  \
lev1_name lev2_name           
Other     MEP         CA-ON   
          MEP         CA-NL   

                                                               level_1  \
lev1_name lev2_name                                                      
Other     MEP        Fish, crustaceans, shellfish and other fishery...   
          MEP        Fish, crustaceans, shellfish and other fishery...   

                     A1-A4_intermediate_output_('CA-ON', 'Gross fixed capital formation, Construction', 'Residential structures')_kg CO2 eq (short)  \
lev1_name lev2_name                                                                                                                                   
Other     MEP                                                13.957133                                                                                
          MEP                                                13.557572                                                                                

                     A5_final_demand_('CA-ON', 'Gross fixed capital formation, Construction', 'Residential structures')_kg CO2 eq (short)  \
lev1_name lev2_name                                                                                                                         
Other     MEP                                                      0.0                                                                      
          MEP                                                      0.0                                                                      

                    country_province  \
lev1_name lev2_name                    
Other     MEP                  CA-ON   
          MEP                  CA-NL   

                                                             commodity  lev1  \
lev1_name lev2_name                                                            
Other     MEP        Fish, crustaceans, shellfish and other fishery...     6   
          MEP        Fish, crustaceans, shellfish and other fishery...     6   

                        lev2    LCA  
lev1_name lev2_name                  
Other     MEP        04  013  A1-A3  
          MEP        04  013  A1-A3

In [52]:
manufac_serv = lev1_other.loc[lev1_other.lev2_name == 'Other' ]
manufac_serv.head(50)


,level_0,level_1,"A1-A4_intermediate_output_('CA-ON', 'Gross fixed capital formation, Construction', 'Residential structures')_kg CO2 eq (short)","A5_final_demand_('CA-ON', 'Gross fixed capital formation, Construction', 'Residential structures')_kg CO2 eq (short)",country_province,commodity,lev1,lev1_name,lev2,lev2_name,LCA
new_index,,,,,,,,,,,
CA-ON Wholesale margins - building materials and supplies,CA-ON,Wholesale margins - building materials and sup...,2.406768e+08,0.0,CA-ON,Wholesale margins - building materials and sup...,6,Other,06 01,Other,0
"CA-ON Retail margins - building materials, garden equipment and supplies",CA-ON,"Retail margins - building materials, garden eq...",1.293835e+08,0.0,CA-ON,"Retail margins - building materials, garden eq...",6,Other,06 01,Other,0
"CA-ON Floriculture and nursery products (except cannabis), and sod",CA-ON,Floriculture and nursery products (except cann...,9.617138e+07,0.0,CA-ON,Floriculture and nursery products (except cann...,6,Other,06 01,Other,A1-A3
CA-ON Paperboard containers,CA-ON,Paperboard containers,5.215750e+07,0.0,CA-ON,Paperboard containers,6,Other,06 01,Other,A1-A3
CA-ON Retail margins - furniture and home furnishings,CA-ON,Retail margins - furniture and home furnishings,4.254059e+07,0.0,CA-ON,Retail margins - furniture and home furnishings,6,Other,06 01,Other,0
CA-ON Wholesale margins - personal and household goods,CA-ON,Wholesale margins - personal and household goods,2.427888e+07,0.0,CA-ON,Wholesale margins - personal and household goods,6,Other,06 01,Other,0
CA-ON Printed products,CA-ON,Printed products,2.359897e+07,0.0,CA-ON,Printed products,6,Other,06 01,Other,A1-A3
CA-ON Other converted paper products,CA-ON,Other converted paper products,2.338615e+07,0.0,CA-ON,Other converted paper products,6,Other,06 01,Other,A1-A3
"CA-ON Carpets, rugs and mats",CA-ON,"Carpets, rugs and mats",1.543876e+07,0.0,CA-ON,"Carpets, rugs and mats",6,Other,06 01,Other,A1-A3


In [50]:
manufac_serv.to_csv(expath + tests+'on_res_material_Mnufacturing_services.csv')


PermissionError: [Errno 13] Permission denied: 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/tests/on_res_material_Mnufacturing_services.csv'

In [6]:
x.to_csv(expath + tests+'on_res_sub_material_class_blue05.csv')

In [ ]:
left.sum()

(CA-ON, Gross fixed capital formation, Construction, Construction)  kg CO2 eq (short)    6.505901e+07
dtype: float64

## 1.1 Run IO for Ontario GFCFC

In [3]:
# find zero sum sectors and remove from list
for s in sub_sec_list:
    final_demand_category = (province, sector, s)
    if tables_Y[(province, sector, s)].sum() == 0:
        print ('in '+province + ', - '+ s + " is empty")
        sub_sec_list.remove(s)
        print (s +'removed from list' )


NameError: name 'sub_sec_list' is not defined

In [41]:
# Run IO calculation on lists and append to a table

# Create an empty GFCFC DF for joining loop calculations
on_gfcfc_sub_materials = pd.DataFrame(index=tables_Y.index)

for s in sub_sec_list:
    # Designate final demand sector
    # this demo uses ('CA-ON', 'Gross fixed capital formation, Construction')
    #define province /sector /sub sector
    province = 'CA-ON'
    sector = 'Gross fixed capital formation, Construction'
    sub_sector = 'Construction'

    # this parameter will propegate as a headar to the env impact table
    final_demand_category = (province, sector, s)
    final_demand_category

    # isolate final demnad vector

    on_gfcf_sub_Y = tables_Y[(final_demand_category)]
    #transform series into df
    on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
    on_gfcf_sub_Y
    print ('claculating ', s)
    # 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
    on_gfcf_sub_AY= vector_X(tables_A, on_gfcf_sub_Y)
    #call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df

    # This line multiplies (L)16Kx16K * (AYdiag) 16Kx16K - the intermediate output

    on_gfcf_sub_D= vector_D(tables_L, on_gfcf_sub_AY[1])
    on_gfcf_sub_D

    #2. Calculate environmental impact. returnes kgCO2e for called total output vector  
    # call [1] for diagonal or [0] sector summary
    on_gfcf_sub_CSD = env_impact(tables_CS, on_gfcf_sub_D[0])

    #3. assign title to column
    on_gfcf_sub_CSD = pd.DataFrame(on_gfcf_sub_CSD).rename(
        columns = {'Climate change, short term': ('on_gfcfc_'+ s)})
    on_gfcf_sub_CSD

    #4. calculating final demand emissions LCA const. process (A4-A5). (CS)(Y)diag
    #4.1 diagonolize Y
    on_gfcf_sub_Ydiag = pd.DataFrame(np.diag(on_gfcf_sub_Y.sum(1)), index=on_gfcf_sub_Y.index, columns= on_gfcf_sub_Y.index)
    #on_res_struct_Ydiag
    #multiply final demand vector with env impact.
    on_gfcf_sub_CSY = env_impact(tables_CS, on_gfcf_sub_Ydiag)
    on_gfcf_sub_CSY = pd.DataFrame(on_gfcf_sub_CSY).rename(
        columns = {'Climate change, short term':'on_gfcfc_'+ s})
    #on_res_struct_CSY.sum()
    #this line tests the intermediate output and final demand sum up to the sector's total emissions.
    #on_res_struct_CSD.sum()+on_res_struct_CSY.sum()
    on_gfcf_sub_CSY
    print (s, ' calculates successfuly')

    #5. Join vectors and into DF.
    left = on_gfcf_sub_CSD
    right = on_gfcf_sub_CSY
    on_gfcf_sub_commod_LCA = left.join(right, lsuffix = '_A1-A4', rsuffix = '_A5_finl_dmnd')

    on_gfcf_sub_commod_LCA

    #appending result to DF
    left = on_gfcfc_sub_materials
    right = on_gfcf_sub_commod_LCA
    on_gfcfc_sub_materials = left.join(right, rsuffix = s)

    print (s, 'vector appended')
print ('dataframe created sussessfuly')

claculating  Accommodation and food services
Accommodation and food services  calculates successfuly
Accommodation and food services vector appended
claculating  Administrative and support, waste management and remediation services


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_20392\35751484.py:67: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  on_gfcfc_sub_materials = left.join(right, rsuffix = s)


Administrative and support, waste management and remediation services  calculates successfuly
Administrative and support, waste management and remediation services vector appended
claculating  Animal production


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_20392\35751484.py:67: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  on_gfcfc_sub_materials = left.join(right, rsuffix = s)


Animal production  calculates successfuly
Animal production vector appended
claculating  Arts, entertainment and recreation


...

Finance and insurance  calculates successfuly
Finance and insurance vector appended
claculating  Fishing, hunting and trapping


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_20392\35751484.py:67: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  on_gfcfc_sub_materials = left.join(right, rsuffix = s)


Fishing, hunting and trapping  calculates successfuly
Fishing, hunting and trapping vector appended
claculating  Food manufacturing


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_20392\35751484.py:67: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  on_gfcfc_sub_materials = left.join(right, rsuffix = s)


KeyboardInterrupt: 

In [10]:
on_gfcfc_sub_materials.to_csv(expath + tests + 'test_on_gfcfc_sub_CO2e.csv')

In [16]:
sec_sum =  on_gfcfc_sub_materials.sum()
sec_sum.sum()

26823712264.380802

# 2.0 Create a concotanated df of all province materials and sources by Construction Sector

In [22]:
# Upload emission DF for each province and concatanate them into a commodityXgfcf sector (200KX108) matrix

In [3]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
# 1.Create an empty Canada DF for joining loop calculations
# 1.1. load files (material by residential)
prov_gfcfc_mat = pd.read_csv(ca_material_lib +  'CA-ONgfcfc_sub_materials.csv')
prov_gfcfc_mat.head(2)
#ca_res_mat_emission = pd.DataFrame(index = prov_gfcfc_mat.index)
ca_gscfc_mat_emission = prov_gfcfc_mat[['Unnamed: 0','Unnamed: 1']]
ca_gscfc_mat_emission

# 1.2 rename column names
col_names = pd.read_csv(ca_material_lib +  provinces[1]+'gfcfc_sub_materials.csv')
new_column_name = col_names.columns.str[8:]
new_column_name
#col_names.columns


########
# 2. load province materials from librarty
prov_sector_commods = []
for p in provinces:
    # 2. load files (material by residential)
    x = pd.read_csv(ca_material_lib +  p+'gfcfc_sub_materials.csv')
    #ca_material_lib +  p+'gfcfc_sub_materials.csv'
    print(p,'  file loaded')
    ## REWRITE column names
    x.columns = x.columns.str[8:]

    # store x
    prov_sector_commods.append(x)
    
# concat all x
prov_sector_commods_joined = pd.concat(prov_sector_commods)
# group by country and material - Canadawide gfcfc 
prov_sector_commods_joined.groupby([' 0', ' 1']).sum().reset_index()

#######

#3. map the material classification df (using join)
prov_sector_commods_joined

# # 3.1. import calssification
path = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/lists/'
class_list = pd.read_csv(path + 'material_lca_classification09.csv')
# arrange by country/commodity
class_list.set_index(['country_province', 'commodity' ])

# removes multi-index from LCA DF     left = on_res_commod_LCA.droplevel([1], axis = 1)
left = prov_sector_commods_joined

#Creating a refernce index for both DF (country+commodity)
    # left = left.reset_index()
left['new_index'] = left[' 0'] + ' ' + left[' 1']
left = left.set_index('new_index')
left

#######

# 4. Attach material list funciton
right = class_list
right['new_index'] = right['country_province'] + ' ' + right['commodity']
right = right.set_index('new_index')
right
#Joining DF
material_df = left.join(right)

######

# 5. Group by material classificaiton
x= material_df
x = x.groupby(['lev1_name', 'lev2_name']).sum()
#ca_gfcfc_mat_emission = x.drop(['index'], axis = 1)
ca_gfcfc_mat_emission = x.drop(['lev1'], axis = 1)


ca_gfcfc_mat_emission


CA-NL   file loaded
CA-PE   file loaded
CA-NS   file loaded
CA-NB   file loaded
CA-QC   file loaded
CA-ON   file loaded
CA-MB   file loaded
CA-SK   file loaded
CA-AB   file loaded
CA-BC   file loaded
CA-YT   file loaded
CA-NT   file loaded
CA-NU   file loaded


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_21652\624023672.py:72: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = x.groupby(['lev1_name', 'lev2_name']).sum()


gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                               
ConstructionSector DirectEmissions                                                         0.000000e+00                    
Energy             Energy                                                                  1.965957e+07                    
Materials          Asphalt                                                                 9.093810e+06                    
                   Concrete                                                                1.964531e+08                    
                   EarthworkAndLandscaping_Materials                                       3.683414e+07                    
                   Finishes                                                                1.780277e+07                    
                   MEP                                                                     6.146507e+07                    
                   Masonry                                                                 1.296176e+07                    
                   Metals                                                                  1.047777e+08                    
                   Openings                                                                8.343144e+06                    
                   PlasticsAndComposites                                                   2.818076e+07                    
                   ThermalAndMoistureProtection                                            1.851767e+07                    
                   Wood                                                                    3.248297e+07                    
Other              Other                                                                   3.174128e+07                    
Services           DesignServicesAndMargins                                                6.448498e+07                    
                   ManufacturingServices                                                   4.729456e+07                    
Transport          Transport                                                               3.632012e+07                    

                                                      gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                      
ConstructionSector DirectEmissions                                                         3.696286e+07                           
Energy             Energy                                                                  0.000000e+00                           
Materials          Asphalt                                                                 0.000000e+00                           
                   Concrete                                                                0.000000e+00                           
                   EarthworkAndLandscaping_Materials                                       0.000000e+00                           
                   Finishes                                                                0.000000e+00                           
                   MEP                                                                     0.000000e+00                           
                   Masonry                                                                 0.000000e+00                           
                   Metals                                                                  0.000000e+00                           
                   Openings                                                                0.000000e+00                           
                   PlasticsAndComposites                                                   0.000000e+00                      

In [5]:
ca_gfcfc_mat_emission.to_csv(expath+tests+'CA_gfcfc_material_by_sector_A1-A5_class_09.csv')

In [9]:
# find emission which are manufacturing services and CSY
df= material_df
df=df.reset_index(drop=True)
index = (df.iloc[:91] != 0).idxmax()
index
value = df.iloc[index, :]


In [10]:
value.sum()

 0                                                                                                          ATATATATATATATATATATATATATATATATATATATATATATAT...
 1                                                                                                          Additives/Blending ComponentsAdditives/Blendin...
gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')                                                                                 678.241579
gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')                                                                                 0.0
gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')                                            32.740291
                                                                                                                                  ...                        
lev1                                                

In [6]:
# Sum A1-A4 and A5
df = ca_gfcfc_mat_emission
df.iloc[:, 1::2] = df.iloc[:, 1::2].add(df.iloc[:, ::2].values)
df

gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                               
ConstructionSector DirectEmissions                                                         0.000000e+00                    
Energy             Energy                                                                  1.965957e+07                    
Materials          Asphalt                                                                 9.093810e+06                    
                   Concrete                                                                1.964531e+08                    
                   EarthworkAndLandscaping_Materials                                       3.683414e+07                    
                   Finishes                                                                1.780277e+07                    
                   MEP                                                                     6.146507e+07                    
                   Masonry                                                                 1.296176e+07                    
                   Metals                                                                  1.047777e+08                    
                   Openings                                                                8.343144e+06                    
                   PlasticsAndComposites                                                   2.818076e+07                    
                   ThermalAndMoistureProtection                                            1.851767e+07                    
                   Wood                                                                    3.248297e+07                    
Other              Other                                                                   3.174128e+07                    
Services           DesignServicesAndMargins                                                6.448498e+07                    
                   ManufacturingServices                                                   4.729456e+07                    
Transport          Transport                                                               3.632012e+07                    

                                                      gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                      
ConstructionSector DirectEmissions                                                         3.696286e+07                           
Energy             Energy                                                                  1.965957e+07                           
Materials          Asphalt                                                                 9.093810e+06                           
                   Concrete                                                                1.964531e+08                           
                   EarthworkAndLandscaping_Materials                                       3.683414e+07                           
                   Finishes                                                                1.780277e+07                           
                   MEP                                                                     6.146507e+07                           
                   Masonry                                                                 1.296176e+07                           
                   Metals                                                                  1.047777e+08                           
                   Openings                                                                8.343144e+06                           
                   PlasticsAndComposites                                                   2.818076e+07                      

### 2.1.2 find top 10  commoditiesfor each emission type

In [12]:
# This code isolates the top 10 emitting sources from the DF

In [7]:
# The 10 largest commodities emissions
material_df['commod_sum'] = material_df.sum(axis=1)
top_10_rows = material_df.groupby('lev2_name').apply(lambda x: x.nlargest(10, 'commod_sum')).reset_index(drop=True)
gfcfc_material_top_10_commod = top_10_rows[['lev2_name',' 0',' 1','commod_sum']]
gfcfc_material_top_10_commod.to_csv(expath + tests +'gfcfc_material_top_10_commod.csv')
gfcfc_material_top_10_commod


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_8208\1158796428.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  material_df['commod_sum'] = material_df.sum(axis=1)


,lev2_name,0,1,commod_sum
0,Asphalt,CA-ON,Asphalt (except natural) and asphalt products,7.575351e+08
1,Asphalt,CA-AB,Asphalt (except natural) and asphalt products,4.954316e+08
2,Asphalt,CA-QC,Asphalt (except natural) and asphalt products,3.913251e+08
3,Asphalt,CA-SK,Asphalt (except natural) and asphalt products,2.366086e+08
4,Asphalt,CA-AB,Asphalt (except natural) and asphalt products,1.834890e+08
...,...,...,...,...
165,Wood,CA-ON,Softwood lumber,3.383939e+08
166,Wood,CA-AB,Softwood lumber,3.147020e+08
167,Wood,CA-AB,"Wood products, n.e.c.",2.752514e+08
168,Wood,CA-BC,Other sawmill products and treated wood products,2.263333e+08


In [8]:
gfcfc_material_top_10_commod.to_csv(expath+tests+'CA_gfcfc_gfcfc_material_top_10_commod_class_09.csv')

## 2.1.3 Create a Material - by province DF [13 Xprovince, gfcfc sectors] - joined DF

In [11]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
ca_material_lib = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/CA_materials_GFCFC_df/'

# Create an empty Canada DF for joining loop calculations
# 2. load files (material by residential)
prov_gfcfc_mat = pd.read_csv(ca_material_lib +  'CA-ONgfcfc_sub_materials.csv')
prov_gfcfc_mat.head(2)
#ca_res_mat_emission = pd.DataFrame(index = prov_gfcfc_mat.index)
ca_all_gfcfc_mat_emission = prov_gfcfc_mat[['Unnamed: 0','Unnamed: 1']]
ca_all_gfcfc_mat_emission


# 2. load province materials from librarty
for p in provinces:
    # 2. load files (material by residential)
    x = pd.read_csv(ca_material_lib +  p+'gfcfc_sub_materials.csv')
    #ca_material_lib +  p+'gfcfc_sub_materials.csv'
    print(p,'  file loaded')
    #3. isolate all_gfcfc A1-3 A5 columns,
    # for isolating a specific sector use columns location e.g.[:, [90,91]] for residentials
    x_all_gfcfc = x.iloc[:, 2:]
    #add prov res to canada province DF
    ca_all_gfcfc_mat_emission = ca_all_gfcfc_mat_emission.join(x_all_gfcfc, rsuffix=('_'+p))
ca_all_gfcfc_mat_emission


CA-NL   file loaded
CA-PE   file loaded
CA-NS   file loaded
CA-NB   file loaded
CA-QC   file loaded
CA-ON   file loaded
CA-MB   file loaded
CA-SK   file loaded
CA-AB   file loaded
CA-BC   file loaded
CA-YT   file loaded
CA-NT   file loaded
CA-NU   file loaded


,Unnamed: 0,Unnamed: 1,"('CA-NL_gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Administrative and support, waste management and remediation services_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Animal production_A1-A4', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Animal production_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Arts, entertainment and recreation_A1-A4', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Arts, entertainment and recreation_A5_finl_dmnd', 'kg CO2 eq (short)')",...,"('CA-NU_gfcfc_Transportation and warehousing_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Transportation and warehousing_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Transportation equipment manufacturing_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Transportation equipment manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Utilities_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Utilities_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Wholesale trade_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Wholesale trade_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Wood product manufacturing_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Wood product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')"
0,CA-AB,Canola (including rapeseed),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,CA-AB,Oilseeds (except canola),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,CA-AB,Wheat,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,CA-AB,Grains (except wheat),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,CA-AB,Fresh potatoes,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16191,WM,Membership organisation services n.e.c. (91),1.606320,0.0,0.034712,0.0,0.423535,0.0,0.622647,0.0,...,0.048972,0.0,0.0,0.0,0.001254,0.0,0.006253,0.0,0.0,0.0
16192,WM,"Recreational, cultural and sporting services (92)",3.269554,0.0,0.022438,0.0,0.278874,0.0,0.676349,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
16193,WM,Other services (93),0.177465,0.0,0.001614,0.0,0.026282,0.0,0.047518,0.0,...,0.000141,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
16194,WM,Private households with employed persons (95),0.001172,0.0,0.000008,0.0,0.000100,0.0,0.000242,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


In [12]:

# 4. Attach material list funciton
x=ca_all_gfcfc_mat_emission
x= attach_mat_class(ca_all_gfcfc_mat_emission)
#x.head(2)


# 5. Group by material classificaiton
x = x.groupby(['lev1_name', 'lev2_name']).sum()
ca_all_gfcfc_mat_emission = x.drop(['index'], axis = 1)
ca_all_gfcfc_mat_emission = ca_all_gfcfc_mat_emission.drop(['lev1'], axis = 1)
ca_all_gfcfc_mat_emission

C:\Users\Hatzav\AppData\Local\Temp\ipykernel_21652\630794658.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = x.groupby(['lev1_name', 'lev2_name']).sum()


('CA-NL_gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                       
ConstructionSector DirectEmissions                                                         0.000000e+00                            
Energy             Energy                                                                  2.489926e+05                            
Materials          Asphalt                                                                 2.163134e+05                            
                   Concrete                                                                5.319342e+06                            
                   EarthworkAndLandscaping_Materials                                       1.714589e+06                            
                   Finishes                                                                5.159063e+05                            
                   MEP                                                                     2.287413e+06                            
                   Masonry                                                                 3.707176e+05                            
                   Metals                                                                  2.587262e+06                            
                   Openings                                                                5.620300e+04                            
                   PlasticsAndComposites                                                   7.162615e+05                            
                   ThermalAndMoistureProtection                                            2.603918e+05                            
                   Wood                                                                    1.581238e+06                            
Other              Other                                                                   8.486372e+05                            
Services           DesignServicesAndMargins                                                1.706262e+06                            
                   ManufacturingServices                                                   1.774493e+06                            
Transport          Transport                                                               1.020760e+06                            

                                                      ('CA-NL_gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                              
ConstructionSector DirectEmissions                                                         1.048937e+06                                   
Energy             Energy                                                                  0.000000e+00                                   
Materials          Asphalt                                                                 0.000000e+00                                   
                   Concrete                                                                0.000000e+00                                   
                   EarthworkAndLandscaping_Materials                                       0.000000e+00                                   
                   Finishes                                                                0.000000e+00                                   
                   MEP                                                                     0.000000e+00                                   
                   Masonry                                                                 0.000000e+00                                   
                   Metals                                                                  0.000000e+00                                   
                

In [11]:
ca_all_gfcfc_mat_emission.to_csv(expath+tests+'CA_gfcfc_mat_emissions_commod_class_09.csv')

In [25]:
# create a Canada - wide DF
df = ca_all_gfcfc_mat_emission

# Select even columns
even_columns = df.iloc[:, 1::2]

# Select odd columns
odd_columns = df.iloc[:, ::2]

# Calculate the sum of even columns
df1_A1_A4 = even_columns.sum(axis=1)

# Calculate the sum of odd columns
df2_A5 = odd_columns.sum(axis=1)

# Create DataFrame df3 by concatenating df1_A1_A4 and df2_A5 on the right
df3 = pd.concat([df1_A1_A4, df2_A5], axis=1)
df3.columns = ['df1_A1_A4', 'df2_A5']

# You can then reset the index if needed
df3.reset_index(inplace=True)
ca_A1_A5_by_material = df3
ca_A1_A5_by_material

In [26]:
ca_A1_A5_by_material

,lev1_name,lev2_name,df1_A1_A4,df2_A5
0,ConstructionSector,DirectEmissions,8.672411e+09,0.000000e+00
1,Energy,Energy,0.000000e+00,8.151716e+09
2,Materials,Asphalt,0.000000e+00,3.208707e+09
3,Materials,Concrete,0.000000e+00,1.297641e+10
4,Materials,EarthworkAndLandscaping_Materials,0.000000e+00,1.093442e+09
5,Materials,Finishes,0.000000e+00,2.203432e+09
6,Materials,MEP,0.000000e+00,6.272167e+09
7,Materials,Masonry,0.000000e+00,1.394337e+09
8,Materials,Metals,0.000000e+00,1.076157e+10
9,Materials,Openings,0.000000e+00,1.353062e+09


In [27]:
ca_A1_A5_by_material.to_csv(expath+tests+'ca_A1_A5_by_material.csv')

### 2.1.3.1 Create Res. Structures Material distribution- by province DF [13 Xprovince, gfcfc - construction, residenrial structure] - joined DF

In [58]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
ca_material_lib = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/CA_materials_GFCFC_df/'

# Create an empty Canada DF for joining loop calculations
# 2. load files (material by residential)
prov_gfcfc_mat = pd.read_csv(ca_material_lib +  'CA-ONgfcfc_sub_materials.csv')
prov_gfcfc_mat.head(2)
#ca_res_mat_emission = pd.DataFrame(index = prov_gfcfc_mat.index)
ca_all_res_mat_emission = prov_gfcfc_mat[['Unnamed: 0','Unnamed: 1']]
ca_all_res_mat_emission


,Unnamed: 0,Unnamed: 1
0,CA-AB,Canola (including rapeseed)
1,CA-AB,Oilseeds (except canola)
2,CA-AB,Wheat
3,CA-AB,Grains (except wheat)
4,CA-AB,Fresh potatoes
...,...,...
16191,WM,Membership organisation services n.e.c. (91)
16192,WM,"Recreational, cultural and sporting services (92)"
16193,WM,Other services (93)
16194,WM,Private households with employed persons (95)


In [59]:

# 2. load province materials from librarty
for p in provinces:
    # 2. load files (material by residential)
    x = pd.read_csv(ca_material_lib +  p+'gfcfc_sub_materials.csv')
    #ca_material_lib +  p+'gfcfc_sub_materials.csv'
    print(p,'  file loaded')
    #3. isolate all_gfcfc A1-3 A5 columns,
    # for isolating a specific sector use columns location e.g.[:, [89,90]] for residentials
    x_all_res = x.iloc[:, [90,91]]
    #add prov res to canada province DF
    ca_all_res_mat_emission = ca_all_res_mat_emission.join(x_all_res, rsuffix=('_'+p))
ca_all_res_mat_emission


CA-NL   file loaded
CA-PE   file loaded
CA-NS   file loaded
CA-NB   file loaded
CA-QC   file loaded
CA-ON   file loaded
CA-MB   file loaded
CA-SK   file loaded
CA-AB   file loaded
CA-BC   file loaded
CA-YT   file loaded
CA-NT   file loaded
CA-NU   file loaded


,Unnamed: 0,Unnamed: 1,"('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-PE_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-PE_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NS_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NS_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NB_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NB_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')",...,"('CA-AB_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-AB_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-BC_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-BC_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-YT_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-YT_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NT_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NT_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')"
0,CA-AB,Canola (including rapeseed),0.072431,0.0,0.003810,0.0,0.292847,0.0,0.338646,0.0,...,41.877165,0.0,18.710743,0.0,0.332839,0.0,0.098127,0.0,0.376907,0.0
1,CA-AB,Oilseeds (except canola),0.193028,0.0,0.002121,0.0,0.372427,0.0,0.171477,0.0,...,57.227633,0.0,5.407848,0.0,0.088008,0.0,0.044849,0.0,0.014870,0.0
2,CA-AB,Wheat,3.618481,0.0,1.432896,0.0,20.214484,0.0,34.093493,0.0,...,616.372800,0.0,9360.084448,0.0,7.430076,0.0,2.110641,0.0,2.478629,0.0
3,CA-AB,Grains (except wheat),3.005959,0.0,1.157073,0.0,16.655414,0.0,24.241979,0.0,...,699.505248,0.0,10937.214932,0.0,8.690759,0.0,2.430593,0.0,2.879821,0.0
4,CA-AB,Fresh potatoes,0.178512,0.0,0.046486,0.0,0.342846,0.0,0.445304,0.0,...,27.824514,0.0,3.521808,0.0,0.069994,0.0,0.031778,0.0,0.019295,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16191,WM,Membership organisation services n.e.c. (91),60.130105,0.0,19.303821,0.0,196.908756,0.0,113.730303,0.0,...,779.757986,0.0,936.814144,0.0,1.620009,0.0,1.729246,0.0,1.572779,0.0
16192,WM,"Recreational, cultural and sporting services (92)",290.693507,0.0,233.694973,0.0,595.693094,0.0,449.851625,0.0,...,3811.243731,0.0,11513.676050,0.0,15.564870,0.0,13.308833,0.0,7.694465,0.0
16193,WM,Other services (93),16.099977,0.0,11.462912,0.0,58.974193,0.0,39.598786,0.0,...,326.135018,0.0,723.641403,0.0,3.144091,0.0,2.245745,0.0,0.985163,0.0
16194,WM,Private households with employed persons (95),0.377245,0.0,0.211049,0.0,0.948818,0.0,0.890605,0.0,...,8.717885,0.0,9.778233,0.0,0.296829,0.0,0.096184,0.0,0.038172,0.0


In [44]:
print(ca_all_res_mat_emission.sum())

Unnamed: 0                                                                  CA-ABCA-ABCA-ABCA-ABCA-ABCA-ABCA-ABCA-ABCA-ABC...
Unnamed: 1                                                                  Canola (including rapeseed)Oilseeds (except ca...
('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')                                            256169009.924894
('CA-NL_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')                                      27767375.373824
('CA-PE_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')                                            107562132.286045
('CA-PE_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')                                       9722672.207184
('CA-NS_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')                                            917982160.381681
('CA-NS_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')                                       5763706

In [60]:
# 4. Attach material list funciton
x=ca_all_res_mat_emission
x= attach_mat_class(ca_all_res_mat_emission)
x.head(2)


,index,Unnamed: 0,Unnamed: 1,"('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-PE_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-PE_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NS_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NS_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NB_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')",...,"('CA-NT_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')",country_province,commodity,lev1,lev1_name,lev2,lev2_name,LCA
new_index,,,,,,,,,,,,,,,,,,,,,
CA-AB Canola (including rapeseed),0,CA-AB,Canola (including rapeseed),0.072431,0.0,0.003810,0.0,0.292847,0.0,0.338646,...,0.0,0.376907,0.0,CA-AB,Canola (including rapeseed),6,Other,06 01,Other,A1-A3
CA-AB Oilseeds (except canola),1,CA-AB,Oilseeds (except canola),0.193028,0.0,0.002121,0.0,0.372427,0.0,0.171477,...,0.0,0.014870,0.0,CA-AB,Oilseeds (except canola),6,Other,06 01,Other,A1-A3


In [61]:
# 5. Group by material classificaiton
x = x.groupby(['lev1_name', 'lev2_name']).sum()
ca_all_res_mat_emission = x.drop(['index'], axis = 1)
ca_all_res_mat_emission = ca_all_res_mat_emission.drop(['lev1'], axis = 1)
ca_all_res_mat_emission

C:\Users\Hatzav\AppData\Local\Temp\ipykernel_8208\619329204.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = x.groupby(['lev1_name', 'lev2_name']).sum()


('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                              
ConstructionSector DirectEmissions                                                         0.000000e+00                   
Energy             Energy                                                                  7.971070e+06                   
Materials          Asphalt                                                                 4.473097e+06                   
                   Concrete                                                                2.399055e+07                   
                   EarthworkAndLandscaping_Materials                                       4.553460e+06                   
                   Finishes                                                                1.084512e+07                   
                   MEP                                                                     1.946788e+07                   
                   Masonry                                                                 3.304561e+06                   
                   Metals                                                                  2.435139e+07                   
                   Openings                                                                3.643358e+06                   
                   PlasticsAndComposites                                                   1.192907e+07                   
                   ThermalAndMoistureProtection                                            2.984343e+06                   
                   Wood                                                                    5.947180e+07                   
Other              Other                                                                   1.207501e+07                   
Services           DesignServicesAndMargins                                                2.331617e+07                   
                   ManufacturingServices                                                   1.796320e+07                   
Transport          Transport                                                               2.582895e+07                   

                                                      ('CA-NL_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                     
ConstructionSector DirectEmissions                                                         2.045662e+07                          
Energy             Energy                                                                  0.000000e+00                          
Materials          Asphalt                                                                 0.000000e+00                          
                   Concrete                                                                0.000000e+00                          
                   EarthworkAndLandscaping_Materials                                       0.000000e+00                          
                   Finishes                                                                0.000000e+00                          
                   MEP                                                                     0.000000e+00                          
                   Masonry                                                                 0.000000e+00                          
                   Metals                                                                  0.000000e+00                          
                   Openings                                                                0.000000e+00                          
                   PlasticsAndComposites                                                   0.000000e+00                          
                   Thermal

In [62]:
# sum A1-A4 and A5 to one column
sum_df = sum_col(ca_all_res_mat_emission)
sum_df


('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                              
ConstructionSector DirectEmissions                                                         2.045662e+07                   
Energy             Energy                                                                  7.971070e+06                   
Materials          Asphalt                                                                 4.473097e+06                   
                   Concrete                                                                2.399055e+07                   
                   EarthworkAndLandscaping_Materials                                       4.553460e+06                   
                   Finishes                                                                1.084512e+07                   
                   MEP                                                                     1.946788e+07                   
                   Masonry                                                                 3.304561e+06                   
                   Metals                                                                  2.435139e+07                   
                   Openings                                                                3.643358e+06                   
                   PlasticsAndComposites                                                   1.192907e+07                   
                   ThermalAndMoistureProtection                                            2.984343e+06                   
                   Wood                                                                    5.947180e+07                   
Other              Other                                                                   1.207501e+07                   
Services           DesignServicesAndMargins                                                3.062692e+07                   
                   ManufacturingServices                                                   1.796320e+07                   
Transport          Transport                                                               2.582895e+07                   

                                                      ('CA-PE_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                              
ConstructionSector DirectEmissions                                                         8.414213e+06                   
Energy             Energy                                                                  7.982395e+06                   
Materials          Asphalt                                                                 2.372038e+06                   
                   Concrete                                                                1.154498e+07                   
                   EarthworkAndLandscaping_Materials                                       1.185122e+06                   
                   Finishes                                                                4.250998e+06                   
                   MEP                                                                     7.263769e+06                   
                   Masonry                                                                 2.183943e+06                   
                   Metals                                                                  1.115299e+07                   
                   Openings                                                                1.894026e+06                   
                   PlasticsAndComposites                                                   3.918189e+06                   
                   ThermalAndMoistureProtection                                            2.073349e+06              

In [63]:
#clean column names
n= sum_df.rename(columns=lambda x: x[2: ])
n= n.rename(columns=lambda x: x[:-29 ])
ca_all_res_mat_emission = n
ca_all_res_mat_emission

CA-NL_gfcfc_Residential structures  \
lev1_name          lev2_name                                                               
ConstructionSector DirectEmissions                                          2.045662e+07   
Energy             Energy                                                   7.971070e+06   
Materials          Asphalt                                                  4.473097e+06   
                   Concrete                                                 2.399055e+07   
                   EarthworkAndLandscaping_Materials                        4.553460e+06   
                   Finishes                                                 1.084512e+07   
                   MEP                                                      1.946788e+07   
                   Masonry                                                  3.304561e+06   
                   Metals                                                   2.435139e+07   
                   Openings                                                 3.643358e+06   
                   PlasticsAndComposites                                    1.192907e+07   
                   ThermalAndMoistureProtection                             2.984343e+06   
                   Wood                                                     5.947180e+07   
Other              Other                                                    1.207501e+07   
Services           DesignServicesAndMargins                                 3.062692e+07   
                   ManufacturingServices                                    1.796320e+07   
Transport          Transport                                                2.582895e+07   

                                                      CA-PE_gfcfc_Residential structures  \
lev1_name          lev2_name                                                               
ConstructionSector DirectEmissions                                          8.414213e+06   
Energy             Energy                                                   7.982395e+06   
Materials          Asphalt                                                  2.372038e+06   
                   Concrete                                                 1.154498e+07   
                   EarthworkAndLandscaping_Materials                        1.185122e+06   
                   Finishes                                                 4.250998e+06   
                   MEP                                                      7.263769e+06   
                   Masonry                                                  2.183943e+06   
                   Metals                                                   1.115299e+07   
                   Openings                                                 1.894026e+06   
                   PlasticsAndComposites                                    3.918189e+06   
                   ThermalAndMoistureProtection                             2.073349e+06   
                   Wood                                                     2.317768e+07   
Other              Other                                                    4.634400e+06   
Services           DesignServicesAndMargins                                 9.429688e+06   
                   ManufacturingServices                                    6.950164e+06   
Transport          Transport                                                8.856865e+06   

                                                      CA-NS_gfcfc_Residential structures  \
lev1_name          lev2_name                                                               
ConstructionSector DirectEmissions                                          4.723344e+07   
Energy             Energy                                                   5.682850e+07   
Materials          Asphalt                                                  1.893218e+07   
                   Concrete                                                 1.464291e+08   
    

In [48]:
ca_all_res_mat_emission.sum(axis=1)

lev1_name           lev2_name                        
ConstructionSector  DirectEmissions                      2.148877e+09
Energy              Energy                               1.320999e+09
Materials           Asphalt                              9.560291e+08
                    Concrete                             4.683181e+09
                    EarthworkAndLandscaping_Materials    3.073229e+08
                    Finishes                             1.696579e+09
                    MEP                                  2.283510e+09
                    Masonry                              4.733675e+08
                    Metals                               3.253567e+09
                    Openings                             1.063513e+09
                    PlasticsAndComposites                1.818194e+09
                    ThermalAndMoistureProtection         7.564579e+08
                    Wood                                 7.242797e+09
Other               Other           

In [47]:
ca_all_res_mat_emission.to_csv(expath+tests+'CA_materials_by_all_province_gfcfc_constrution_resStructures_17X13_class_09.csv')


## 2.1.4 Create a lean material and commodity list for all sub sectors

In [13]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
# 1.Create an empty Canada DF for joining loop calculations
# 1.1. load files (material by residential)
prov_gfcfc_mat = pd.read_csv(ca_material_lib +  'CA-ONgfcfc_sub_materials.csv')
prov_gfcfc_mat.head(2)
#ca_res_mat_emission = pd.DataFrame(index = prov_gfcfc_mat.index)
ca_gscfc_mat_emission = prov_gfcfc_mat[['Unnamed: 0','Unnamed: 1']]
ca_gscfc_mat_emission

# 1.2 rename column names
col_names = pd.read_csv(ca_material_lib +  provinces[1]+'gfcfc_sub_materials.csv')
new_column_name = col_names.columns.str[8:]
new_column_name
#col_names.columns


########
# 2. load province materials from librarty
prov_sector_commods = []
for p in provinces:
    # 2. load files (material by residential)
    x = pd.read_csv(ca_material_lib +  p+'gfcfc_sub_materials.csv')
    #ca_material_lib +  p+'gfcfc_sub_materials.csv'
    print(p,'  file loaded')
    ## REWRITE column names
    x.columns = x.columns.str[8:]

    # store x
    prov_sector_commods.append(x)
    
# concat all x
prov_sector_commods_joined = pd.concat(prov_sector_commods)
# group by country and material - Canadawide gfcfc 
prov_sector_commods_joined.groupby([' 0', ' 1']).sum().reset_index()


CA-NL   file loaded
CA-PE   file loaded
CA-NS   file loaded
CA-NB   file loaded
CA-QC   file loaded
CA-ON   file loaded
CA-MB   file loaded
CA-SK   file loaded
CA-AB   file loaded
CA-BC   file loaded
CA-YT   file loaded
CA-NT   file loaded
CA-NU   file loaded


,0,1,"gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Animal production_A1-A4', 'kg CO2 eq (short)')","gfcfc_Animal production_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A1-A4', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A5_finl_dmnd', 'kg CO2 eq (short)')",...,"gfcfc_Transportation and warehousing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Transportation and warehousing_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Transportation equipment manufacturing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Transportation equipment manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Utilities_A1-A4', 'kg CO2 eq (short)')","gfcfc_Utilities_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Wholesale trade_A1-A4', 'kg CO2 eq (short)')","gfcfc_Wholesale trade_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Wood product manufacturing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Wood product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')"
0,AT,Additives/Blending Components,4.311678e-08,0.0,5.196326e-09,0.0,1.545943e-08,0.0,2.846096e-08,0.0,...,3.986654e-08,0.0,2.647447e-09,0.0,6.510692e-09,0.0,1.557481e-08,0.0,2.672830e-09,0.0
1,AT,Air transport services (62),9.156578e+03,0.0,1.323749e+03,0.0,4.211895e+03,0.0,5.714951e+03,0.0,...,1.869493e+04,0.0,8.627071e+02,0.0,1.399421e+04,0.0,4.300085e+03,0.0,3.683695e+02,0.0
2,AT,Aluminium and aluminium products,4.983644e+02,0.0,1.617126e+02,0.0,2.299458e+02,0.0,4.206198e+02,0.0,...,7.629553e+03,0.0,5.005683e+01,0.0,8.675352e+03,0.0,2.985912e+02,0.0,5.858825e+01,0.0
3,AT,Aluminium ores and concentrates,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0
4,AT,Animal products nec,2.116388e-04,0.0,2.550617e-05,0.0,7.588265e-05,0.0,1.397007e-04,0.0,...,1.956850e-04,0.0,1.299500e-05,0.0,3.195774e-05,0.0,7.644898e-05,0.0,1.311959e-05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16191,ZA,"Wood material for treatment, Re-processing of ...",0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0
16192,ZA,Wood waste for treatment: incineration,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0
16193,ZA,Wood waste for treatment: landfill,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0
16194,ZA,"Wool, silk-worm cocoons",1.049093e-08,0.0,1.264341e-09,0.0,3.761502e-09,0.0,6.924961e-09,0.0,...,9.700101e-09,0.0,6.441619e-10,0.0,1.584145e-09,0.0,3.789575e-09,0.0,6.503380e-10,0.0


In [14]:

#######

#3. map the material classification df (using join)
prov_sector_commods_joined

# # 3.1. import calssification
path = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/lists/'
class_list = pd.read_csv(path + 'material_lca_classification09_lean.csv')
# arrange by country/commodity
class_list.set_index(['country_province', 'commodity' ])


lev1  \
country_province commodity                                                 
CA-AB            Canola (including rapeseed)                           6   
                 Oilseeds (except canola)                              6   
                 Wheat                                                 6   
                 Grains (except wheat)                                 6   
                 Fresh potatoes                                        6   
...                                                                  ...   
WM               Membership organisation services n.e.c. (91)          6   
                 Recreational, cultural and sporting services (92)     6   
                 Other services (93)                                   6   
                 Private households with employed persons (95)         6   
                 Extra-territorial organizations and bodies            6   

                                                                   lev1_name  \
country_province commodity                                                     
CA-AB            Canola (including rapeseed)                           Other   
                 Oilseeds (except canola)                              Other   
                 Wheat                                                 Other   
                 Grains (except wheat)                                 Other   
                 Fresh potatoes                                        Other   
...                                                                      ...   
WM               Membership organisation services n.e.c. (91)          Other   
                 Recreational, cultural and sporting services (92)     Other   
                 Other services (93)                                   Other   
                 Private households with employed persons (95)         Other   
                 Extra-territorial organizations and bodies            Other   

                                                                     lev2  \
country_province commodity                                                  
CA-AB            Canola (including rapeseed)                        06 01   
                 Oilseeds (except canola)                           06 01   
                 Wheat                                              06 01   
                 Grains (except wheat)                              06 01   
                 Fresh potatoes                                     06 01   
...                                                                   ...   
WM               Membership organisation services n.e.c. (91)       06 01   
                 Recreational, cultural and sporting services (92)  06 01   
                 Other services (93)                                06 01   
                 Private households with employed persons (95)      06 01   
                 Extra-territorial organizations and bodies         06 01   

                                                                   lev2_name  \
country_province commodity                                                     
CA-AB            Canola (including rapeseed)                           Other   
                 Oilseeds (except canola)                              Other   
                 Wheat                                                 Other   
                 Grains (except wheat)                                 Other   
                 Fresh potatoes                                        Other   
...                                                                      ...   
WM               Membership organisation services n.e.c. (91)          Other   
                 Recreational, cultural and sporting services (92)     Other   
                 Other services (93)                                   Other   
                 Private households with employed persons (95)         Other   
                 Extra-territorial organizations and bodies            Other

In [15]:

# removes multi-index from LCA DF     left = on_res_commod_LCA.droplevel([1], axis = 1)
left = prov_sector_commods_joined

#Creating a refernce index for both DF (country+commodity)
    # left = left.reset_index()
left['new_index'] = left[' 0'] + ' ' + left[' 1']
left = left.set_index('new_index')
left

#######

# 4. Attach material list funciton
right = class_list
right['new_index'] = right['country_province'] + ' ' + right['commodity']
right = right.set_index('new_index')
right
#Joining DF
material_df = left.join(right)

######

# 5. Group by material classificaiton
x= material_df
# this is the original class differentiating between lev 1 and lev 2
#x = x.groupby(['lev1_name', 'lev2_name']).sum()
x = x.groupby(['lev2_name']).sum()
#ca_gfcfc_mat_emission = x.drop(['index'], axis = 1)
ca_gfcfc_mat_emission = x.drop(['lev1'], axis = 1)


ca_gfcfc_mat_emission

C:\Users\Hatzav\AppData\Local\Temp\ipykernel_24080\2289887165.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = x.groupby(['lev2_name']).sum()


,"gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Animal production_A1-A4', 'kg CO2 eq (short)')","gfcfc_Animal production_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A1-A4', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Beverage and tobacco product manufacturing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Beverage and tobacco product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')",...,"gfcfc_Transportation and warehousing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Transportation and warehousing_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Transportation equipment manufacturing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Transportation equipment manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Utilities_A1-A4', 'kg CO2 eq (short)')","gfcfc_Utilities_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Wholesale trade_A1-A4', 'kg CO2 eq (short)')","gfcfc_Wholesale trade_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Wood product manufacturing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Wood product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')"
lev2_name,,,,,,,,,,,,,,,,,,,,,
Asphalt,9.093810e+06,0.000000e+00,1.688767e+06,0.000000e+00,3.399492e+06,0.000000e+00,1.870410e+07,0.000000e+00,7.758269e+05,0.000000e+00,...,5.917768e+08,0.000000e+00,1.050173e+06,0.000000e+00,1.552980e+07,0.000000e+00,7.265661e+06,0.000000e+00,1.732326e+06,0.000000e+00
Concrete,1.964531e+08,0.000000e+00,4.404064e+07,0.000000e+00,8.322695e+07,0.000000e+00,1.302006e+08,0.000000e+00,1.439834e+07,0.000000e+00,...,1.141541e+09,0.000000e+00,2.492589e+07,0.000000e+00,7.188316e+08,0.000000e+00,8.467789e+07,0.000000e+00,2.413650e+07,0.000000e+00
DesignServicesAndMargins,6.448498e+07,8.076353e+06,1.233834e+07,1.020349e+06,2.698066e+07,3.827065e+06,4.067128e+07,3.925085e+06,3.960992e+06,3.376384e+05,...,5.231356e+08,7.934162e+06,6.959434e+06,4.622855e+05,4.286302e+08,1.530307e+06,3.665207e+07,4.185704e+06,7.102882e+06,3.102568e+05
DirectEmissions,0.000000e+00,3.696286e+07,0.000000e+00,1.861051e+07,0.000000e+00,2.580049e+07,0.000000e+00,5.692774e+07,0.000000e+00,2.504218e+06,...,0.000000e+00,9.119546e+08,0.000000e+00,6.347900e+06,0.000000e+00,1.276619e+09,0.000000e+00,2.430853e+07,0.000000e+00,4.387438e+06
EarthworkAndLandscaping_Materials,3.683414e+07,0.000000e+00,3.725424e+06,0.000000e+00,1.289335e+07,0.000000e+00,1.365287e+07,0.000000e+00,2.487490e+06,0.000000e+00,...,6.887418e+07,0.000000e+00,3.086941e+06,0.000000e+00,8.753538e+06,0.000000e+00,1.641968e+07,0.000000e+00,4.433502e+06,0.000000e+00
Energy,1.965957e+07,0.000000e+00,9.262758e+06,0.000000e+00,1.228092e+07,0.000000e+00,2.450844e+07,0.000000e+00,1.154480e+06,0.000000e+00,...,1.294860e+09,0.000000e+00,2.362328e+06,0.000000e+00,5.186502e+08,0.000000e+00,1.972005e+07,0.000000e+00,3.429738e+06,0.000000e+00
Finishes,1.780277e+07,0.000000e+00,2.394368e+06,0.000000e+00,6.853692e+06,0.000000e+00,8.714896e+06,0.000000e+00,1.181671e+06,0.000000e+00,...,4.384256e+07,0.000000e+00,2.127394e+06,0.000000e+00,1.826581e+07,0.000000e+00,8.388695e+06,0.000000e+00,1.647030e+06,0.000000e+00
MEP,6.146507e+07,0.000000e+00,1.095013e+07,0.000000e+00,2.518731e+07,0.000000e+00,3.678787e+07,0.000000e+00,4.029500e+06,0.000000e+00,...,3.375917e+08,0.000000e+00,7.741801e+06,0.000000e+00,8.002902e+08,0.000000e+00,2.990460e+07,0.000000e+00,6.311338e+06,0.000000e+00
ManufacturingServices,4.729456e+07,0.000000e+00,1.721294e+07,0.000000e+00,2.944905e+07,0.000000e+00,4.742426e+07,0.000000e+00,2.505746e+06,0.000000e+00,...,4.030971e+08,0.000000e+00,5.392257e+06,0.000000e+00,2.507551e+08,0.000000e+00,2.620258e+07,0.000000e+00,4.615433e+06,0.000000e+00


In [7]:
'''
n= ca_gfcfc_mat_emission.rename(columns=lambda x: x[6: ])
n= n.rename(columns=lambda x: x[:-29 ])
ca_gfcfc_mat_emission = n
ca_gfcfc_mat_emission
'''

'\nn= ca_gfcfc_mat_emission.rename(columns=lambda x: x[6: ])\nn= n.rename(columns=lambda x: x[:-29 ])\nca_gfcfc_mat_emission = n\nca_gfcfc_mat_emission\n'

In [8]:
ca_gfcfc_mat_emission.to_csv(expath+tests+'CA_materials_by_all_province_gfcfc_constrution_sub_materials_17X108_class_09_lean.csv')

In [9]:
def sum_col(df):


    # Get the column names
    column_names = df.columns

    # Initialize a new DataFrame to store the summed values
    summed_df = pd.DataFrame()

    # Iterate through even columns (0, 2, 4, etc.) and sum with the following odd column
    for i in range(0, len(column_names) - 1, 2):
        even_column = column_names[i]
        odd_column = column_names[i + 1]
        
        # Sum the even and odd columns and store the result in the new DataFrame
        #summed_df[f'{even_column} + {odd_column}'] = df[even_column] + df[odd_column]
        summed_df[even_column] = df[even_column] + df[odd_column]


    # Display the resulting DataFrame
    return summed_df

x = sum_col(ca_gfcfc_mat_emission)


In [10]:
# rename headars
n= x.rename(columns=lambda x: x[6: ])
n= n.rename(columns=lambda x: x[:-29 ])
ca_all_gfcfc_mat_emission=n
ca_all_gfcfc_mat_emission

,Accommodation and food services,"Administrative and support, waste management and remediation services",Animal production,"Arts, entertainment and recreation",Beverage and tobacco product manufacturing,Chemical manufacturing,Clothing and leather and allied product manufacturing,Computer and electronic product manufacturing,Construction,Crop production,...,Residential structures,Retail trade,Support activities for agriculture and forestry,"Support activities for mining, and oil and gas extraction",Textile and textile product mills,Transportation and warehousing,Transportation equipment manufacturing,Utilities,Wholesale trade,Wood product manufacturing
lev2_name,,,,,,,,,,,,,,,,,,,,,
Asphalt,9.093810e+06,1.688767e+06,3.399492e+06,1.870410e+07,7.758269e+05,4.790668e+06,5.092174e+04,1.151437e+05,3.213286e+06,6.848607e+06,...,9.560291e+08,1.232750e+07,4.843529e+05,5.043268e+05,1.481649e+05,5.917768e+08,1.050173e+06,1.552980e+07,7.265661e+06,1.732326e+06
Concrete,1.964531e+08,4.404064e+07,8.322695e+07,1.302006e+08,1.439834e+07,9.261271e+07,1.061176e+06,2.673387e+06,7.092616e+07,1.440053e+08,...,4.683181e+09,2.320382e+08,2.409068e+06,8.841859e+06,3.387747e+06,1.141541e+09,2.492589e+07,7.188316e+08,8.467789e+07,2.413650e+07
DesignServicesAndMargins,7.256133e+07,1.335869e+07,3.080772e+07,4.459637e+07,4.298630e+06,4.288481e+07,4.909307e+05,8.786105e+05,2.769002e+07,5.618497e+07,...,3.468397e+09,9.402430e+07,8.518586e+05,4.362295e+06,9.461219e+05,5.310698e+08,7.421720e+06,4.301605e+08,4.083777e+07,7.413139e+06
DirectEmissions,3.696286e+07,1.861051e+07,2.580049e+07,5.692774e+07,2.504218e+06,1.824590e+07,2.897438e+05,5.155613e+05,1.232402e+07,4.078595e+07,...,2.148877e+09,5.212803e+07,7.366392e+05,4.601899e+06,5.482515e+05,9.119546e+08,6.347900e+06,1.276619e+09,2.430853e+07,4.387438e+06
EarthworkAndLandscaping_Materials,3.683414e+07,3.725424e+06,1.289335e+07,1.365287e+07,2.487490e+06,2.056511e+07,2.126722e+05,3.756563e+05,1.357015e+07,2.464601e+07,...,3.073229e+08,4.172571e+07,3.851917e+05,1.277382e+06,4.979735e+05,6.887418e+07,3.086941e+06,8.753538e+06,1.641968e+07,4.433502e+06
Energy,1.965957e+07,9.262758e+06,1.228092e+07,2.450844e+07,1.154480e+06,1.228382e+07,1.303116e+05,2.331361e+05,7.129259e+06,1.895128e+07,...,1.320999e+09,3.376656e+07,2.657514e+05,2.702059e+06,2.302424e+05,1.294860e+09,2.362328e+06,5.186502e+08,1.972005e+07,3.429738e+06
Finishes,1.780277e+07,2.394368e+06,6.853692e+06,8.714896e+06,1.181671e+06,1.016428e+07,1.304999e+05,2.498333e+05,6.699862e+06,1.363591e+07,...,1.696579e+09,2.169492e+07,2.340394e+05,6.049853e+05,2.451539e+05,4.384256e+07,2.127394e+06,1.826581e+07,8.388695e+06,1.647030e+06
MEP,6.146507e+07,1.095013e+07,2.518731e+07,3.678787e+07,4.029500e+06,3.372761e+07,4.481811e+05,8.636341e+05,2.263144e+07,4.801446e+07,...,2.283510e+09,7.569021e+07,7.807437e+05,3.725504e+06,9.604288e+05,3.375917e+08,7.741801e+06,8.002902e+08,2.990460e+07,6.311338e+06
ManufacturingServices,4.729456e+07,1.721294e+07,2.944905e+07,4.742426e+07,2.505746e+06,2.773284e+07,3.249815e+05,5.122356e+05,1.686950e+07,4.603032e+07,...,2.275876e+09,6.097062e+07,4.999244e+05,9.169430e+06,4.664311e+05,4.030971e+08,5.392257e+06,2.507551e+08,2.620258e+07,4.615433e+06


In [11]:
# combine columns into other columns
x= ca_all_gfcfc_mat_emission
selected_columns = x.iloc[:, list(range(1,11))+ list(range(13,21)) + [24] + list(range(26,29))
                          + [32] + list(range(36,42))+list(range(46,48))+[50]+list(range(52,53))]
x['others_column'] = selected_columns.sum(axis=1)
# Leave top 15 emitting sectors + others
lean_df3= x.drop(x.iloc[:, list(range(1,12))+ list(range(13,22)) + [24] + list(range(26,30))
                          + [32] + list(range(36,43))+list(range(46,49))+[50]+list(range(52,54))], axis=1)

lean_df3


,Accommodation and food services,Educational services (public),Hospitals,Information and cultural industries,Mining and quarrying (except oil and gas),Oil and gas extraction (except oil sands),Oil sands extraction,Other federal government services (except defence),Other municipal government services,Other provincial and territorial government services,Real estate and rental and leasing,Residential structures,Retail trade,Transportation and warehousing,Utilities,others_column
lev2_name,,,,,,,,,,,,,,,,
Asphalt,9.093810e+06,2.870894e+07,1.818791e+07,4.631770e+06,1.494614e+07,1.357274e+08,4.954724e+07,9.604708e+07,6.172002e+08,5.338979e+08,1.586538e+07,9.560291e+08,1.232750e+07,5.917768e+08,1.552980e+07,1.015788e+08
Concrete,1.964531e+08,6.355270e+08,2.921247e+08,9.319628e+07,4.045606e+08,1.522331e+08,4.210460e+07,2.776239e+08,1.544468e+09,7.841068e+08,3.583704e+08,4.683181e+09,2.320382e+08,1.141541e+09,7.188316e+08,1.268061e+09
DesignServicesAndMargins,7.256133e+07,2.025621e+08,9.144718e+07,1.598879e+08,1.316735e+08,7.292435e+08,3.087002e+08,6.155539e+07,4.698345e+08,2.471053e+08,1.324717e+08,3.468397e+09,9.402430e+07,5.310698e+08,4.301605e+08,5.043491e+08
DirectEmissions,3.696286e+07,1.199376e+08,5.512247e+07,4.931241e+08,3.736291e+08,7.292904e+08,2.415742e+08,1.429216e+08,1.046248e+09,5.449604e+08,7.048394e+07,2.148877e+09,5.212803e+07,9.119546e+08,1.276619e+09,3.975949e+08
EarthworkAndLandscaping_Materials,3.683414e+07,1.102899e+08,5.242599e+07,1.163553e+07,1.903266e+07,9.853887e+05,1.738980e+06,1.562378e+07,8.699657e+07,4.848637e+07,5.995096e+07,3.073229e+08,4.172571e+07,6.887418e+07,8.753538e+06,1.963022e+08
Energy,1.965957e+07,4.490609e+07,2.162951e+07,1.725546e+08,1.659424e+08,2.759785e+09,8.862725e+08,4.568871e+07,3.777320e+08,2.008369e+08,3.476075e+07,1.320999e+09,3.376656e+07,1.294860e+09,5.186502e+08,2.383111e+08
Finishes,1.780277e+07,6.025654e+07,2.580037e+07,1.929805e+07,8.252684e+06,5.356688e+06,2.342615e+06,1.152201e+07,5.878163e+07,3.548833e+07,3.622856e+07,1.696579e+09,2.169492e+07,4.384256e+07,1.826581e+07,1.285711e+08
MEP,6.146507e+07,2.059665e+08,8.929606e+07,6.380465e+08,1.215618e+08,3.955230e+08,1.435959e+08,5.255966e+07,3.248622e+08,1.547059e+08,1.213542e+08,2.283510e+09,7.569021e+07,3.375917e+08,8.002902e+08,4.170564e+08
ManufacturingServices,4.729456e+07,1.158142e+08,5.113574e+07,9.101260e+07,4.405072e+08,4.114469e+08,1.781441e+08,7.129614e+07,6.505786e+08,2.189330e+08,8.352242e+07,2.275876e+09,6.097062e+07,4.030971e+08,2.507551e+08,3.893925e+08


In [12]:
df2 = lean_df3.groupby(['lev2_name']).sum()
ca_all_gfcfc_mat_emission = df2
ca_all_gfcfc_mat_emission


,Accommodation and food services,Educational services (public),Hospitals,Information and cultural industries,Mining and quarrying (except oil and gas),Oil and gas extraction (except oil sands),Oil sands extraction,Other federal government services (except defence),Other municipal government services,Other provincial and territorial government services,Real estate and rental and leasing,Residential structures,Retail trade,Transportation and warehousing,Utilities,others_column
lev2_name,,,,,,,,,,,,,,,,
Asphalt,9.093810e+06,2.870894e+07,1.818791e+07,4.631770e+06,1.494614e+07,1.357274e+08,4.954724e+07,9.604708e+07,6.172002e+08,5.338979e+08,1.586538e+07,9.560291e+08,1.232750e+07,5.917768e+08,1.552980e+07,1.015788e+08
Concrete,1.964531e+08,6.355270e+08,2.921247e+08,9.319628e+07,4.045606e+08,1.522331e+08,4.210460e+07,2.776239e+08,1.544468e+09,7.841068e+08,3.583704e+08,4.683181e+09,2.320382e+08,1.141541e+09,7.188316e+08,1.268061e+09
DesignServicesAndMargins,7.256133e+07,2.025621e+08,9.144718e+07,1.598879e+08,1.316735e+08,7.292435e+08,3.087002e+08,6.155539e+07,4.698345e+08,2.471053e+08,1.324717e+08,3.468397e+09,9.402430e+07,5.310698e+08,4.301605e+08,5.043491e+08
DirectEmissions,3.696286e+07,1.199376e+08,5.512247e+07,4.931241e+08,3.736291e+08,7.292904e+08,2.415742e+08,1.429216e+08,1.046248e+09,5.449604e+08,7.048394e+07,2.148877e+09,5.212803e+07,9.119546e+08,1.276619e+09,3.975949e+08
EarthworkAndLandscaping_Materials,3.683414e+07,1.102899e+08,5.242599e+07,1.163553e+07,1.903266e+07,9.853887e+05,1.738980e+06,1.562378e+07,8.699657e+07,4.848637e+07,5.995096e+07,3.073229e+08,4.172571e+07,6.887418e+07,8.753538e+06,1.963022e+08
Energy,1.965957e+07,4.490609e+07,2.162951e+07,1.725546e+08,1.659424e+08,2.759785e+09,8.862725e+08,4.568871e+07,3.777320e+08,2.008369e+08,3.476075e+07,1.320999e+09,3.376656e+07,1.294860e+09,5.186502e+08,2.383111e+08
Finishes,1.780277e+07,6.025654e+07,2.580037e+07,1.929805e+07,8.252684e+06,5.356688e+06,2.342615e+06,1.152201e+07,5.878163e+07,3.548833e+07,3.622856e+07,1.696579e+09,2.169492e+07,4.384256e+07,1.826581e+07,1.285711e+08
MEP,6.146507e+07,2.059665e+08,8.929606e+07,6.380465e+08,1.215618e+08,3.955230e+08,1.435959e+08,5.255966e+07,3.248622e+08,1.547059e+08,1.213542e+08,2.283510e+09,7.569021e+07,3.375917e+08,8.002902e+08,4.170564e+08
ManufacturingServices,4.729456e+07,1.158142e+08,5.113574e+07,9.101260e+07,4.405072e+08,4.114469e+08,1.781441e+08,7.129614e+07,6.505786e+08,2.189330e+08,8.352242e+07,2.275876e+09,6.097062e+07,4.030971e+08,2.507551e+08,3.893925e+08


In [30]:
ca_all_gfcfc_mat_emission.to_csv(expath+tests+'CA_materials_by_all_province_gfcfc_constrution_sub_materials_Lean_11X54.csv')


## 2.1.5 GFCFC by province

In [99]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
ca_material_lib = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/CA_materials_GFCFC_df/'



In [69]:
def sum_odd_even_columns_with_original_names(dataframe):
    columns = dataframe.columns
    sum_df = pd.DataFrame()

    for i in range(0, len(columns), 2):
        if i + 1 < len(columns):
            col_name = f'Sum_{columns[i]}'
            sum_series = dataframe[columns[i]] + dataframe[columns[i + 1]]
            sum_df[col_name] = sum_series

    return sum_df

In [102]:
# 2. load province materials from librarty

ca_gfcfc_sectors_by_prov = pd.DataFrame()

for p in provinces:
    # 2. load files (material by residential)
    x = pd.read_csv(ca_material_lib +  p+'gfcfc_sub_materials.csv')
    #ca_material_lib +  p+'gfcfc_sub_materials.csv'
    print(p,'  file loaded')
    #3. isolate all_gfcfc A1-3 A5 columns,
    # for isolating a specific sector use columns location e.g.[:, [90,91]] for residentials
    x_all_gfcfc = pd.DataFrame(x.iloc[:, 2:])
    x_all_gfcfc
    prov = sum_odd_even_columns_with_original_names(x_all_gfcfc)
    prov
    #trim column name
    n= prov.rename(columns=lambda x: x[18: ])
    n= n.rename(columns=lambda x: x[:-29 ])
    prov = pd.DataFrame(n.sum()).rename(columns={0 : p})
    prov
    ca_gfcfc_sectors_by_prov = pd.concat([ca_gfcfc_sectors_by_prov, prov], axis = 1)
ca_gfcfc_sectors_by_prov.head(5)


CA-NL   file loaded
CA-PE   file loaded
CA-NS   file loaded
CA-NB   file loaded
CA-QC   file loaded
CA-ON   file loaded
CA-MB   file loaded
CA-SK   file loaded
CA-AB   file loaded
CA-BC   file loaded
CA-YT   file loaded
CA-NT   file loaded
CA-NU   file loaded


,CA-NL,CA-PE,CA-NS,CA-NB,CA-QC,CA-ON,CA-MB,CA-SK,CA-AB,CA-BC,CA-YT,CA-NT,CA-NU
Accommodation and food services,2.236052e+07,7.637803e+06,2.620867e+07,1.146541e+07,1.458028e+08,1.900838e+08,2.256340e+07,2.822308e+07,1.645218e+08,1.396582e+08,9.257180e+05,1.864864e+06,1.013663e+07
"Administrative and support, waste management and remediation services",1.679589e+06,5.185699e+05,3.353267e+06,4.440788e+06,3.101918e+07,6.086941e+07,2.775364e+06,4.965066e+06,4.714155e+07,2.122098e+07,7.683737e+04,1.108832e+05,1.829126e+04
Animal production,6.125823e+06,1.666373e+06,1.720646e+07,7.801773e+06,8.297935e+07,8.790851e+07,1.842188e+07,2.431234e+07,7.230677e+07,2.642904e+07,3.132017e+04,0.000000e+00,0.000000e+00
"Arts, entertainment and recreation",1.027281e+07,8.553147e+05,1.264232e+07,6.124801e+06,1.029594e+08,2.205007e+08,2.633206e+07,1.720247e+07,1.055147e+08,5.224937e+07,2.002575e+06,4.238824e+06,3.100074e+05
Beverage and tobacco product manufacturing,3.632744e+05,5.297257e+04,1.769958e+06,6.877774e+05,1.118946e+07,1.407950e+07,2.847932e+06,2.634172e+05,2.235246e+06,1.758525e+07,1.950719e+04,0.000000e+00,2.779463e+05


In [105]:
ca_gfcfc_sectors_by_prov.to_csv(expath+tests+'ca_gfcfc_sectors_by_prov_108X13.csv')

### top five sectors

In [106]:

df = pd.read_csv(expath +  'ca_gfcfc_sectors_by_prov_108X13.csv',index_col = 0)
df #= df.T.head()

# Create an empty list to store the top 5 values from each column
top_5_values = []

# Iterate through each column and extract the top 5 values
for column in df.columns:
    top_5 = df.nlargest(5, column)[column]#.tolist()
    top_5_values.append(top_5)
top_5

# Create a new DataFrame from the list of top 5 values
top_5_df = pd.DataFrame(top_5_values).T
top_5_df

,CA-NL,CA-PE,CA-NS,CA-NB,CA-QC,CA-ON,CA-MB,CA-SK,CA-AB,CA-BC,CA-YT,CA-NT,CA-NU
Oil and gas extraction (except oil sands),6.693365e+08,NaN,NaN,NaN,NaN,NaN,1.571958e+08,1.253203e+09,4.182514e+09,8.812211e+08,NaN,NaN,NaN
Utilities,3.719835e+08,1.098923e+07,7.694785e+07,7.145176e+07,6.344871e+08,1.753301e+09,6.038754e+08,4.039212e+08,NaN,1.002907e+09,NaN,NaN,NaN
Residential structures,2.839364e+08,1.172848e+08,9.756192e+08,4.196089e+08,6.843331e+09,1.345435e+10,1.174126e+09,1.002302e+09,5.755459e+09,7.613598e+09,4.612788e+07,2.786249e+07,9.576524e+06
Mining and quarrying (except oil and gas),1.315935e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.706556e+07,1.025530e+08,2.331274e+08
Other municipal government services,9.633080e+07,8.840690e+06,8.253454e+07,1.601188e+08,1.171189e+09,2.197315e+09,1.884762e+08,4.123890e+08,1.575459e+09,1.217101e+09,1.321214e+07,3.197181e+07,1.386011e+07
Other provincial and territorial government services,NaN,1.852643e+07,8.482443e+07,1.698369e+08,7.479084e+08,1.067338e+09,NaN,NaN,NaN,NaN,1.337257e+07,4.669417e+07,7.056430e+07
Transportation and warehousing,NaN,8.721789e+06,NaN,NaN,6.996333e+08,2.328485e+09,2.668728e+08,7.791060e+08,1.742211e+09,1.228069e+09,NaN,NaN,NaN
Educational services (public),NaN,NaN,8.995449e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Information and cultural industries,NaN,NaN,NaN,4.218467e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oil sands extraction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.197320e+09,NaN,NaN,NaN,NaN


In [ ]:
top_5_df.to_csv(expath+tests+'top_5ca_gfcfc_sectors_by_prov.csv')

## 2.2 Material classification by gfcf construction sub sector (One province only)

In [25]:
#load table without recalculating EEIO
data_path  ='C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/'
on_gfcfc_sub_materials = pd.read_csv(data_path+ 'on_gfcfc_sub_sectors_df_CO2e_blue_01.csv', header=[0], index_col=[0,1])
on_gfcfc_sub_materials

#join materials to sub sector - output is a TUPLE [0,1,2]
# # 1. import calssification
path = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/lists/'
class_list = pd.read_csv(path +  'material_lca_classification09.csv')
# arrange by country/commodity
class_list.set_index(['country_province','commodity' ])

# removes multi-index from LCA DF     left = on_res_commod_LCA.droplevel([1], axis = 1)
left = on_gfcfc_sub_materials
#Creating a refernce index for both DF (country+commodity)
left = left.reset_index()
left['new_index'] = left['level_0'] + ' ' + left['level_1']
left = left.set_index('new_index')
left

right = class_list
right['new_index'] = right['country_province'] + ' ' + right['commodity']
right = right.set_index('new_index')
right

#Joining DF
material_df = left.join(right)
material_df


,level_0,level_1,"('on_gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')","('on_gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')","('on_gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')","('on_gfcfc_Administrative and support, waste management and remediation services_A5_finl_dmnd', 'kg CO2 eq (short)')","('on_gfcfc_Animal production_A1-A4', 'kg CO2 eq (short)')","('on_gfcfc_Animal production_A5_finl_dmnd', 'kg CO2 eq (short)')","('on_gfcfc_Arts, entertainment and recreation_A1-A4', 'kg CO2 eq (short)')","('on_gfcfc_Arts, entertainment and recreation_A5_finl_dmnd', 'kg CO2 eq (short)')",...,"('on_gfcfc_Wholesale trade_A5_finl_dmnd', 'kg CO2 eq (short)')","('on_gfcfc_Wood product manufacturing_A1-A4', 'kg CO2 eq (short)')","('on_gfcfc_Wood product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')",country_province,commodity,lev1,lev1_name,lev2,lev2_name,LCA
new_index,,,,,,,,,,,,,,,,,,,,,
CA-AB Canola (including rapeseed),CA-AB,Canola (including rapeseed),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,CA-AB,Canola (including rapeseed),6,Other,06 01,Other,A1-A3
CA-AB Oilseeds (except canola),CA-AB,Oilseeds (except canola),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,CA-AB,Oilseeds (except canola),6,Other,06 01,Other,A1-A3
CA-AB Wheat,CA-AB,Wheat,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,CA-AB,Wheat,6,Other,06 01,Other,A1-A3
CA-AB Grains (except wheat),CA-AB,Grains (except wheat),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,CA-AB,Grains (except wheat),6,Other,06 01,Other,A1-A3
CA-AB Fresh potatoes,CA-AB,Fresh potatoes,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,CA-AB,Fresh potatoes,6,Other,06 01,Other,A1-A3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM Membership organisation services n.e.c. (91),WM,Membership organisation services n.e.c. (91),13.651209,0.0,2.731661,0.0,5.662481,0.0,12.003798,0.0,...,0.0,0.181359,0.0,WM,Membership organisation services n.e.c. (91),6,Other,06 01,Other,0
"WM Recreational, cultural and sporting services (92)",WM,"Recreational, cultural and sporting services (92)",214.372142,0.0,30.805295,0.0,82.986294,0.0,149.543624,0.0,...,0.0,1.267794,0.0,WM,"Recreational, cultural and sporting services (92)",6,Other,06 01,Other,0
WM Other services (93),WM,Other services (93),18.174549,0.0,2.732540,0.0,7.097245,0.0,12.986214,0.0,...,0.0,0.120763,0.0,WM,Other services (93),6,Other,06 01,Other,0


In [26]:

#Sorts by material lev 1 lev 2
material_df_sum = material_df.groupby((['lev1_name', 'lev2_name',])).sum().drop('lev1', axis =1 )
material_df_commod = material_df.groupby((['lev1_name', 'lev2_name', 'commodity'])).sum().drop('lev1', axis =1 )
# Append to tuple
on_gfcfc_sub_materials = (material_df, material_df_sum, material_df_commod)


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_20392\2606850201.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  material_df_sum = material_df.groupby((['lev1_name', 'lev2_name',])).sum().drop('lev1', axis =1 )
C:\Users\Hatzav\AppData\Local\Temp\ipykernel_20392\2606850201.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  material_df_commod = material_df.groupby((['lev1_name', 'lev2_name', 'commodity'])).sum().drop('lev1', axis =1 )


In [27]:
on_gfcfc_sub_materials[0]
#Sort by material (0 - full DF, 
# 1 - summmed by material group
# 2- summed by material name
x= on_gfcfc_sub_materials[1]
x.sort_values(by = ['lev2_name']) #, ascending = True).sum(1)

('on_gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                    
Materials          Concrete                                                                4.451337e+07                         
ConstructionSector DirectEmissions                                                         1.396709e+05                         
Materials          EarthworkAndLandscaping_Materials                                       4.311025e+06                         
ConstructionSector Energy                                                                  4.402512e+06                         
Materials          Energy                                                                  1.914005e+05                         
                   ExteriorImprovements                                                    2.531032e+06                         
                   Finishes                                                                4.458990e+06                         
                   MEP                                                                     1.804907e+07                         
Other              Margins                                                                 6.285908e+06                         
Materials          Masonry                                                                 2.295178e+06                         
                   Metals                                                                  3.029111e+07                         
                   Openings                                                                2.314389e+06                         
Other              Other                                                                   1.684218e+06                         
Materials          OtherMaterials                                                          3.121956e+06                         
Services           OtherServices                                                           1.867382e+07                         
Materials          PlasticsAndComposites                                                   7.611712e+06                         
                   Raw_Materials                                                           2.675971e+06                         
                   Specialties                                                             3.775110e+04                         
                   ThermalAndMoistureProtection                                            6.254155e+06                         
Transport          TransportService                                                        9.405410e+06                         
                   Vehicle                                                                 2.256579e+04                         
Water              Water                                                                   4.202277e+04                         
Materials          Wood                                                                    6.144026e+06                         

                                                      ('on_gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                           
Materials          Concrete                                                                0.000000e+00                                
ConstructionSector DirectEmissions                                                         1.265890e+07                                
Materials          EarthworkAndLandscaping_Materials                                       0.000000e+00                                
ConstructionSector Energy                                                                  0.000000e+00                                
Materials   

## 2.3 Canada wide calculation Loop BLUE (all provinces and all GFCFC-sub sectors)

In [4]:
#list provinces and sectors to run in loop
sector = 'Gross fixed capital formation, Construction'
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
sub_sectors = tables_Y[('CA-ON', sector)]
sub_sec_list = sub_sectors.columns.values.tolist()
len(sub_sec_list) # list length is 54 sectors


NameError: name 'tables_Y' is not defined

In [37]:
# Designate final demand sector
# this demo uses ('CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = provinces
sector = 'Gross fixed capital formation, Construction'
sub_sector = sub_sec_list


# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector, sub_sector)
final_demand_category


(['CA-NL',
  'CA-PE',
  'CA-NS',
  'CA-NB',
  'CA-QC',
  'CA-ON',
  'CA-MB',
  'CA-SK',
  'CA-AB',
  'CA-BC',
  'CA-YT',
  'CA-NT',
  'CA-NU'],
 'Gross fixed capital formation, Construction',
 ['Accommodation and food services',
  'Administrative and support, waste management and remediation services',
  'Animal production',
  'Arts, entertainment and recreation',
  'Beverage and tobacco product manufacturing',
  'Chemical manufacturing',
  'Clothing and leather and allied product manufacturing',
  'Computer and electronic product manufacturing',
  'Construction',
  'Crop production',
  'Defence services',
  'Educational services (private)',
  'Educational services (public)',
  'Electrical equipment, appliance and component manufacturing',
  'Fabricated metal product manufacturing',
  'Finance and insurance',
  'Fishing, hunting and trapping',
  'Food manufacturing',
  'Forestry and logging',
  'Furniture and related product manufacturing',
  'Health care and social assistance',
  'Hol

In [38]:
sub_sector

['Accommodation and food services',
 'Administrative and support, waste management and remediation services',
 'Animal production',
 'Arts, entertainment and recreation',
 'Beverage and tobacco product manufacturing',
 'Chemical manufacturing',
 'Clothing and leather and allied product manufacturing',
 'Computer and electronic product manufacturing',
 'Construction',
 'Crop production',
 'Defence services',
 'Educational services (private)',
 'Educational services (public)',
 'Electrical equipment, appliance and component manufacturing',
 'Fabricated metal product manufacturing',
 'Finance and insurance',
 'Fishing, hunting and trapping',
 'Food manufacturing',
 'Forestry and logging',
 'Furniture and related product manufacturing',
 'Health care and social assistance',
 'Holding companies',
 'Hospitals',
 'Information and cultural industries',
 'Machinery manufacturing',
 'Mining and quarrying (except oil and gas)',
 'Miscellaneous manufacturing',
 'Non-metallic mineral product manufa

In [5]:
# create lists to stor data
ca_master_gfcfc_df = pd.DataFrame()
province_gfcfc_list = []

for p in provinces[0:2]:
    for s in sub_sector:
        final_demand_category = (p, sector, s)
        #print ('Caclc ' + str(final_demand_category))
        #print (tables_Y[(p, sector, s)].sum())
        #---- calculationcode goes here ----
         # isolate final demnad vector

        on_gfcf_sub_Y = tables_Y[(final_demand_category)]
        #transform series into df
        on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
        on_gfcf_sub_Y
        print ('in ',p, ' claculating ', s)
        # 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
        on_gfcf_sub_AY= vector_X(tables_A, on_gfcf_sub_Y)
        #call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df

        # This line multiplies (L)16Kx16K * (AYdiag) 16Kx16K - the intermediate output

        on_gfcf_sub_D= vector_D(tables_L, on_gfcf_sub_AY[1])
        on_gfcf_sub_D

        #2. Calculate environmental impact. returnes kgCO2e for called total output vector  
        # call [1] for diagonal or [0] sector summary
        on_gfcf_sub_CSD = env_impact(tables_CS, on_gfcf_sub_D[0])

        #3. assign title to column
        on_gfcf_sub_CSD = pd.DataFrame(on_gfcf_sub_CSD).rename(
            columns = {'Climate change, short term': (p+'_gfcfc_'+ s)})
        on_gfcf_sub_CSD

        #4. calculating final demand emissions LCA const. process (A4-A5). (CS)(Y)diag
        #4.1 diagonolize Y
        on_gfcf_sub_Ydiag = pd.DataFrame(np.diag(on_gfcf_sub_Y.sum(1)), index=on_gfcf_sub_Y.index, columns= on_gfcf_sub_Y.index)
        #on_res_struct_Ydiag
        #multiply final demand vector with env impact.
        on_gfcf_sub_CSY = env_impact(tables_CS, on_gfcf_sub_Ydiag)
        on_gfcf_sub_CSY = pd.DataFrame(on_gfcf_sub_CSY).rename(
            columns = {'Climate change, short term':p+'_gfcfc_'+ s})
        #on_res_struct_CSY.sum()
        #this line tests the intermediate output and final demand sum up to the sector's total emissions.
        #on_res_struct_CSD.sum()+on_res_struct_CSY.sum()
        on_gfcf_sub_CSY
        print ('in' ,p, ' -', s, ' calculates successfuly')

        #5. Join vectors and into DF.
        left = on_gfcf_sub_CSD
        right = on_gfcf_sub_CSY
        on_gfcf_sub_commod_LCA = left.join(right, lsuffix = '_A1-A4', rsuffix = '_A5_finl_dmnd')

        on_gfcf_sub_commod_LCA

        #appending result to DF
        left = on_gfcfc_sub_materials
        right = on_gfcf_sub_commod_LCA
        on_gfcfc_sub_materials = left.join(right, rsuffix = s)

        #------------------------------------
        print ('In ' + p+ ', '+s +'vector appended')
        on_gfcfc_sub_materials
    print ('---- '+p+ ' CALCULATED SUCCESSFULY ---')
    on_gfcfc_sub_materials.to_csv(expath+tests+ p +'gfcfc_sub_materials.csv')
    province_gfcfc_list = province_gfcfc_list.append(on_gfcfc_sub_materials)
    ca_master_gfcfc_df = ca_master_gfcfc_df.join(on_gfcfc_sub_materials, rsuffix = p)
print ('--------------dataframe created sussessfuly-------')    
            
    


NameError: name 'sub_sector' is not defined

In [98]:
ca_master_gfcfc_df


pandas.core.frame.DataFrame

# Tests
github test change

In [54]:
x.to_csv(expath + tests+'on_gfcfc_sub_material_class_blue05.csv')

Isolate category and search for non zero values


In [17]:
material_df

,0,1,"gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Animal production_A1-A4', 'kg CO2 eq (short)')","gfcfc_Animal production_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A1-A4', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A5_finl_dmnd', 'kg CO2 eq (short)')",...,"gfcfc_Wood product manufacturing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Wood product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')",country_province,commodity,lev1,lev1_name,lev2,lev2_name,LCA,commod_sum
new_index,,,,,,,,,,,,,,,,,,,,,
AT Additives/Blending Components,AT,Additives/Blending Components,1.622382e-09,0.0,1.113400e-11,0.0,1.383797e-10,0.0,3.356105e-10,0.0,...,1.431514e-11,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3,4.000002
AT Additives/Blending Components,AT,Additives/Blending Components,3.961265e-11,0.0,3.860555e-12,0.0,1.628147e-11,0.0,1.258877e-11,0.0,...,1.678502e-13,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3,4.000001
AT Additives/Blending Components,AT,Additives/Blending Components,8.880582e-10,0.0,1.166567e-10,0.0,4.732524e-10,0.0,3.090338e-10,0.0,...,2.650214e-11,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3,4.000007
AT Additives/Blending Components,AT,Additives/Blending Components,3.873189e-09,0.0,6.064178e-10,0.0,8.767713e-10,0.0,1.312341e-09,0.0,...,9.743989e-10,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3,4.000005
AT Additives/Blending Components,AT,Additives/Blending Components,8.144679e-09,0.0,5.481935e-10,0.0,4.730392e-09,0.0,3.642032e-09,0.0,...,4.406953e-10,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3,4.000046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZA products of Vegetable oils and fats,ZA,products of Vegetable oils and fats,2.082283e-01,0.0,4.568384e-02,0.0,8.490557e-02,0.0,9.208144e-02,0.0,...,2.045343e-02,0.0,ZA,products of Vegetable oils and fats,6,Other,06 01,Other,0,30.092159
ZA products of Vegetable oils and fats,ZA,products of Vegetable oils and fats,5.944411e-01,0.0,1.225057e-01,0.0,1.179231e-01,0.0,2.795796e-01,0.0,...,1.461406e-01,0.0,ZA,products of Vegetable oils and fats,6,Other,06 01,Other,0,75.804875
ZA products of Vegetable oils and fats,ZA,products of Vegetable oils and fats,7.169024e-06,0.0,2.124155e-07,0.0,0.000000e+00,0.0,1.853325e-05,0.0,...,0.000000e+00,0.0,ZA,products of Vegetable oils and fats,6,Other,06 01,Other,0,6.001847


In [33]:
c = material_df
#c=c.reset_index()


In [34]:
x= c.loc[c['lev2_name'] == 'DesignServicesAndMargins']


In [35]:
y = x.iloc[:,[0,1,90,91]]


In [36]:
y.head()

,0,1,"gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')"
new_index,,,,
AT Other business services (74),AT,Other business services (74),1710.604027,61.334496
AT Other business services (74),AT,Other business services (74),1100.097446,44.473173
AT Other business services (74),AT,Other business services (74),6071.814437,196.179354
AT Other business services (74),AT,Other business services (74),3743.549531,133.655018
AT Other business services (74),AT,Other business services (74),52331.453664,1363.834651


In [37]:
y= y.reset_index()
y


,new_index,0,1,"gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')"
0,AT Other business services (74),AT,Other business services (74),1710.604027,61.334496
1,AT Other business services (74),AT,Other business services (74),1100.097446,44.473173
2,AT Other business services (74),AT,Other business services (74),6071.814437,196.179354
3,AT Other business services (74),AT,Other business services (74),3743.549531,133.655018
4,AT Other business services (74),AT,Other business services (74),52331.453664,1363.834651
...,...,...,...,...,...
5832,ZA Real estate services (70),ZA,Real estate services (70),7.056605,0.003191
5833,ZA Real estate services (70),ZA,Real estate services (70),7.452351,0.008538
5834,ZA Real estate services (70),ZA,Real estate services (70),0.040881,0.002222
5835,ZA Real estate services (70),ZA,Real estate services (70),0.081000,0.000002


In [38]:
print(x.columns)


Index([' 0', ' 1',
       'gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')',
       'gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')',
       'gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')',
       'gfcfc_Administrative and support, waste management and remediation services_A5_finl_dmnd', 'kg CO2 eq (short)')',
       'gfcfc_Animal production_A1-A4', 'kg CO2 eq (short)')',
       'gfcfc_Animal production_A5_finl_dmnd', 'kg CO2 eq (short)')',
       'gfcfc_Arts, entertainment and recreation_A1-A4', 'kg CO2 eq (short)')',
       'gfcfc_Arts, entertainment and recreation_A5_finl_dmnd', 'kg CO2 eq (short)')',
       ...
       'gfcfc_Wood product manufacturing_A1-A4', 'kg CO2 eq (short)')',
       'gfcfc_Wood product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')',
       'country_province', 'commodity', 'lev1', 'lev1_name', 'lev2',
       'lev2_name', 'LCA', 'commod_sum'],
      dtyp

In [40]:
sorted_df = y[y.iloc[:, 2] != 0].sort_values(by=y.columns[2], ascending=False)
sorted_df

,new_index,0,1,"gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')"
2963,CA-NU Wholesale trade commissions,CA-NU,Wholesale trade commissions,2.548219e+02,0.000000
4005,CA-QC Wholesale trade commissions,CA-QC,Wholesale trade commissions,7.388277e+03,0.000000
3313,CA-ON Wholesale trade commissions,CA-ON,Wholesale trade commissions,3.124719e+03,0.000000
3314,CA-ON Wholesale trade commissions,CA-ON,Wholesale trade commissions,4.773387e+02,0.000000
4016,CA-QC Wholesale trade commissions,CA-QC,Wholesale trade commissions,6.314575e+02,0.000000
...,...,...,...,...,...
1218,"CA-NB Architectural, engineering and related s...",CA-NB,"Architectural, engineering and related services",7.661609e+04,4566.440320
1219,"CA-NB Architectural, engineering and related s...",CA-NB,"Architectural, engineering and related services",7.435926e-01,0.000000
1220,"CA-NB Architectural, engineering and related s...",CA-NB,"Architectural, engineering and related services",2.943215e-01,0.000000
1221,"CA-NB Architectural, engineering and related s...",CA-NB,"Architectural, engineering and related services",2.335114e-01,0.000000


In [41]:
sorted_df.iloc[-10:,2]

1213    Architectural, engineering and related services
1214    Architectural, engineering and related services
1215    Architectural, engineering and related services
1216    Architectural, engineering and related services
1217    Architectural, engineering and related services
1218    Architectural, engineering and related services
1219    Architectural, engineering and related services
1220    Architectural, engineering and related services
1221    Architectural, engineering and related services
1560    Architectural, engineering and related services
Name:  1, dtype: object

In [65]:
c.to_csv(expath+tests+'Direct_emissions_construction_work.csv')

In [ ]:
material_df

,0,1,"gfcfc_Accommodation and food services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Accommodation and food services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A1-A4', 'kg CO2 eq (short)')","gfcfc_Administrative and support, waste management and remediation services_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Animal production_A1-A4', 'kg CO2 eq (short)')","gfcfc_Animal production_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A1-A4', 'kg CO2 eq (short)')","gfcfc_Arts, entertainment and recreation_A5_finl_dmnd', 'kg CO2 eq (short)')",...,"gfcfc_Wholesale trade_A5_finl_dmnd', 'kg CO2 eq (short)')","gfcfc_Wood product manufacturing_A1-A4', 'kg CO2 eq (short)')","gfcfc_Wood product manufacturing_A5_finl_dmnd', 'kg CO2 eq (short)')",country_province,commodity,lev1,lev1_name,lev2,lev2_name,LCA
new_index,,,,,,,,,,,,,,,,,,,,,
AT Additives/Blending Components,AT,Additives/Blending Components,1.622382e-09,0.0,1.113400e-11,0.0,1.383797e-10,0.0,3.356105e-10,0.0,...,0.0,1.431514e-11,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3
AT Additives/Blending Components,AT,Additives/Blending Components,3.961265e-11,0.0,3.860555e-12,0.0,1.628147e-11,0.0,1.258877e-11,0.0,...,0.0,1.678502e-13,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3
AT Additives/Blending Components,AT,Additives/Blending Components,8.880582e-10,0.0,1.166567e-10,0.0,4.732524e-10,0.0,3.090338e-10,0.0,...,0.0,2.650214e-11,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3
AT Additives/Blending Components,AT,Additives/Blending Components,3.873189e-09,0.0,6.064178e-10,0.0,8.767713e-10,0.0,1.312341e-09,0.0,...,0.0,9.743989e-10,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3
AT Additives/Blending Components,AT,Additives/Blending Components,8.144679e-09,0.0,5.481935e-10,0.0,4.730392e-09,0.0,3.642032e-09,0.0,...,0.0,4.406953e-10,0.0,AT,Additives/Blending Components,4,Materials,04 01,Concrete,A1-A3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZA products of Vegetable oils and fats,ZA,products of Vegetable oils and fats,2.082283e-01,0.0,4.568384e-02,0.0,8.490557e-02,0.0,9.208144e-02,0.0,...,0.0,2.045343e-02,0.0,ZA,products of Vegetable oils and fats,6,Other,06 01,Other,0
ZA products of Vegetable oils and fats,ZA,products of Vegetable oils and fats,5.944411e-01,0.0,1.225057e-01,0.0,1.179231e-01,0.0,2.795796e-01,0.0,...,0.0,1.461406e-01,0.0,ZA,products of Vegetable oils and fats,6,Other,06 01,Other,0
ZA products of Vegetable oils and fats,ZA,products of Vegetable oils and fats,7.169024e-06,0.0,2.124155e-07,0.0,0.000000e+00,0.0,1.853325e-05,0.0,...,0.0,0.000000e+00,0.0,ZA,products of Vegetable oils and fats,6,Other,06 01,Other,0


### Series of commodities values decending

In [ ]:
tables_Y

CA-AB  \
                                                                                             Changes in inventories   
                                                        Changes in inventories, finished goods and goods in process   
CA-AB Canola (including rapeseed)                                                             7.050776e+07            
      Oilseeds (except canola)                                                               -9.120000e+05            
      Wheat                                                                                   1.158967e+08            
      Grains (except wheat)                                                                  -1.661400e+08            
      Fresh potatoes                                                                          1.879337e+06            
...                                                                                                    ...            
WM    Membership organisation services n.e.c. (91)                                            0.000000e+00            
      Recreational, cultural and sporting services (92)                                       0.000000e+00            
      Other services (93)                                                                     0.000000e+00            
      Private households with employed persons (95)                                           0.000000e+00            
      Extra-territorial organizations and bodies                                              0.000000e+00            

                                                                                                                              \
                                                                                                                               
                                                        Changes in inventories, raw materials and goods purchased for resale   
CA-AB Canola (including rapeseed)                                                             9.689773e+06                     
      Oilseeds (except canola)                                                               -2.600000e+05                     
      Wheat                                                                                   1.061048e+08                     
      Grains (except wheat)                                                                   2.329314e+06                     
      Fresh potatoes                                                                         -4.981000e+06                     
...                                                                                                    ...                     
WM    Membership organisation services n.e.c. (91)                                            0.000000e+00                     
      Recreational, cultural and sporting services (92)                                       0.000000e+00                     
      Other services (93)                                                                     0.000000e+00                     
      Private households with employed persons (95)                                           0.000000e+00                     
      Extra-territorial organizations and bodies                                              0.000000e+00                     

                                                                                                   \
                                                        Governments final consumption expenditure   
                                                                                 Defence services   
CA-AB Canola (including rapeseed)                                                             0.0   
      Oilseeds (except canola)                                                                0.0   
      Wheat                                                                                   0.0   
      Grains (except wheat)                 

In [ ]:
##Commodity Isolation (row) function
# isolate GFCGC for ontario -to create a 1 level column df
province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Construction'

commodity = 'Concrete products'
gfcfc_on_y = tables_Y[('CA-ON' , sector, sub_sector)]
gfcfc_on_y.sort_values(ascending=False).to_csv(expath + tests+'on_Construction_Y_list.csv')
gfcfc_on_y.sort_values(ascending=False)


CA-ON  Industrial buildings (except mine buildings)                       2.059800e+08
       Office buildings                                                   1.099410e+08
       Real estate brokerage and other services related to real estate    1.993569e+07
CA-QC  Real estate brokerage and other services related to real estate    5.523678e+05
CA-MB  Real estate brokerage and other services related to real estate    6.168964e+04
                                                                              ...     
CA-QC  Other federal government services                                  0.000000e+00
       Other provincial and territorial government services               0.000000e+00
       Other municipal government services                                0.000000e+00
       Other aboriginal government services                               0.000000e+00
WM     Extra-territorial organizations and bodies                         0.000000e+00
Name: (CA-ON, Gross fixed capital formation

In [ ]:
on_res_struct_y = tables_Y[('CA-ON' , sector, 'Residential structures')]
on_res_struct_y.sort_values(ascending=False).to_csv(expath + tests+'on_res_struct_Y_list.csv')

In [ ]:
for p in provinces:
    sum = gfcfc_on_y.loc[(p, commodity)].sum(0)
    print (sum)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [ ]:
##Commodity Isolation (row) function
province = 'CA-ON'
commodity = 'Ready-mixed concrete'

#isolate_commodity_to_Y_vector
df = tables_Y[province]
sum_Y = pd.DataFrame(df).sum(1)
sum_Y
#create a 0 sum vector (with Y table rows)
sum_Y_null = pd.Series(0, index=sum_Y.index)
#isolate chosen commodity summary 
sum_Y_null.loc[(province, commodity)] = sum_Y.loc[(province, commodity)]
df =sum_Y_null.sum()
df


0.0

In [ ]:

#this function isolates commodity by province, sums it up and turnes it into a series.
#The function replaces all other vector-commodities with 0 so that when it runs on the model 
# - we only calculate the commodity impact. 
#Isolate Province
df = tables_Y[province]
###create that has the same index and the same columns. all the calues are 0. 

df = pd.DataFrame(df)
df_zero = df.mask(df != 0, 0)
df_zero.sum(1)
#df


CA-AB  Canola (including rapeseed)                          0.0
       Oilseeds (except canola)                             0.0
       Wheat                                                0.0
       Grains (except wheat)                                0.0
       Fresh potatoes                                       0.0
                                                           ... 
WM     Membership organisation services n.e.c. (91)         0.0
       Recreational, cultural and sporting services (92)    0.0
       Other services (93)                                  0.0
       Private households with employed persons (95)        0.0
       Extra-territorial organizations and bodies           0.0
Length: 16196, dtype: float64

In [ ]:
df = tables_Y[province]


0.0

In [ ]:
#isolate chosen commodity summary 
df_zero.loc[(province, commodity)] = df.loc[(province, commodity)]


In [ ]:
#check the commodity is inserted to the df
df_zero.sum(1).sort_values(ascending=False)


CA-AB  Canola (including rapeseed)                                         0.0
PT     Crops nec                                                           0.0
PL     Wood waste for treatment: landfill                                  0.0
       Membership organisation services n.e.c. (91)                        0.0
       Recreational, cultural and sporting services (92)                   0.0
                                                                          ... 
CA-QC  Community college and C.E.G.E.P services provided by governments    0.0
       University services provided by governments                         0.0
       Other educational services provided by governments                  0.0
       Hospital services provided by governments                           0.0
WM     Extra-territorial organizations and bodies                          0.0
Length: 16196, dtype: float64

In [ ]:
df_zero_output = tables_C.dot(tables_S).dot(tables_L).dot(df_zero)
report = df_zero_output.loc[('Climate change, short term','kg CO2 eq (short)')].sort_values(ascending=False)
#exclude sectors and industries with 0
report = report[report !=0]
report